# Cleaning of the Long Covid Data

There are things that I would do directly that should undoubtedly be done in data cleaning like:
* creating multiple data frames to work with each individually at least during data cleaning then maybe group them together later
* unifying data values spelling in text data 
* making sure all columns are of correct data type 
* remove unnessary columns
* dealing with units

etc.

Link to the questionnaire used for the study:
https://drive.google.com/file/d/1tRQRxAntduUmT_AiigQoN95Uk8cMRLgo/view?usp=sharing

Questionnaire sections:
* Demographics and baseline characteristics
* COVID-19 Testing
* COVID-19 experience
* Hospitalization
* Treatments
* DePaul Symptom Questionnaire
* MOS SURVEY (SF-36) 
* Patient Health Questionnaire (PHQ-9)
* Other Symptoms


## 0. Setuping up data wrangling

In [1]:
##imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
#from autocorrect import Speller
import re 
#spell = Speller(lang='en')
import multiprocessing
import time
## remove data between brackets 
def remove_brackets(string):
    
    while "(" in str(string) and ")" in str(string):
        br1=string.index("(")
        br2=string.index(")")
        #print(br1,br2,string[br1:br2+1])
        string=string.replace(string[br1:br2+1],"")
    return string
print(remove_brackets("bla bla bla (hahahaha,ffff) (4444),44,55,66(i)"),remove_brackets(None))


ModuleNotFoundError: No module named 'matplotlib'

In [4]:
## read the data 
data=pd.read_excel("Long covid_ Data collection  (Responses).xlsx",engine="openpyxl")
data.head().T

,0,1,2,3,4
Timestamp,2021-11-21 22:10:42.557000,2021-11-26 22:35:28.161000,2021-11-27 07:22:40.949000,2021-11-27 14:14:40.847000,2021-11-27 14:51:23.895000
Email Address,nona.ali171999@gmail.com,humaaa.juttt@gmail.com,salmana.moneeb@gmail.co,shafaqzahid111@gmail.com,shafaqzahid111@gmail.com
Name of the data collector,Ahmed Khaled,Huma Sadaqat Virk,Quaid e Azam,Shahzaib Ahmad,Shahzaib Ahmad
Name of the institution of the data collector,NaN,King Edward medical University,Lincon in college,King edward medical university lahore,King Edward Medical University Lahore Pakistan
Country of the data collector,Egypt,Pakistan,Pakistan,Pakistan,Pakistan
...,...,...,...,...,...
53.\t Indicate the severity of your vertigo/dizziness (0–10),NaN,2.0,5.0,1.0,6.0
54.\t Have you ever experienced tinnitus before COVID-19 diagnosis?,No,No,No,Yes,No
55. Have you started experiencing tinnitus after diagnosis of COVID-19?,No,No,No,No,No
"56. If yes, please specify the characteristics of your tinnitus",NaN,continuous,NaN,occasional,NaN


In [5]:
# before spliting this huge dataframe into multiple ones let us add one column so
# so that we can have a unique ID for every patient 
data["PatientID"]=np.arange(0,len(data)).astype(str)
data["PatientID"]

0          0
1          1
2          2
3          3
4          4
        ... 
2566    2566
2567    2567
2568    2568
2569    2569
2570    2570
Name: PatientID, Length: 2571, dtype: object

In [6]:
for index,column in enumerate(data.columns):
    print(column,f"\n--- index :{index} ----\n ")

Timestamp 
--- index :0 ----
 
Email Address 
--- index :1 ----
 
Name of the data collector 
--- index :2 ----
 
Name of the institution of the data collector 
--- index :3 ----
 
Country of the data collector 
--- index :4 ----
 
1.	What is your height? 
--- index :5 ----
 
2.	What is your weight? 
--- index :6 ----
 
3.	In which country do you currently reside?  
--- index :7 ----
 
4.	What city do you live in? Please include state if applicable. (i.e. New York, NY) 
--- index :8 ----
 
5.	What type of area do you live in? 	 
--- index :9 ----
 
6.	What age group do you fall into?  
--- index :10 ----
 
7.	Sex	 
--- index :11 ----
 
8.	If applicable, are you pregnant?  
--- index :12 ----
 
10.	If applicable, do you have periods/a menstrual cycle?  
--- index :13 ----
 
11.	Which of the following best describes your ancestry? Select all that apply.  
--- index :14 ----
 
12.	Is your household income from all sources?  
--- index :15 ----
 
13.	What is your highest educational level 

I will create a seperate data for each of the following 
* demographics ----> <mark>  **Demo** </mark>
* covid testing and experience -- > <mark>  **Covid** </mark>
* Treatment , hoptilisation and vaccination ---> <mark>  **treatment** </mark> 
* Each of the following surveys related to symptoms alone:
    * DePaul Symptom Questionnaire ---> <mark>  **symp_1** </mark>
    * MOS SURVEY (SF-36) ---> <mark>  **symp_2** </mark>
    * Patient Health Questionnaire (PHQ-9) ---> <mark>  **symp_3** </mark>
    * Other Symptoms ---> <mark>  **symp_4** </mark>

In [9]:
## 176 is the index of the PatientID column 
Demo= data.iloc[:,np.r_[176,5:19]]
Covid= data.iloc[:,np.r_[176,19:30]]
treatment= data.iloc[:,np.r_[176,30:39]]
symp_1= data.iloc[:,np.r_[176,39:48]]
symp_2= data.iloc[:,np.r_[176,48:112]]
symp_3= data.iloc[:,np.r_[176,112:121]]
sympy_4= data.iloc[:,np.r_[176,121:176]]

In [6]:
# just one last thing to speed things up 

def num_cat(data):
    num_columns=[]
    cat_columns= []
    for i in data.columns:
        if pd.api.types.is_object_dtype(data[i]):
            cat_columns.append(i)
        else:
            num_columns.append(i)
    return num_columns,cat_columns

def full_report(data):
    num,cat=num_cat(data)
    #cat.remove("PatientID")
    print(f"Missing Values\n {data.isna().sum()} \n")
    print(f"Data types \n {data.dtypes} \n")
    print(f" Quick Description \n {data.describe} \n")
    print(f"Value_counts\n")
    for c in cat:
        print(c,data[c].value_counts())
        print("\n")

# 1. Actual Data Wrangling 

## 1 Demographic Data 

### 1. 0 General Look 

In [7]:
Demo.columns

Index(['PatientID', '1.\tWhat is your height?', '2.\tWhat is your weight?',
       '3.\tIn which country do you currently reside? ',
       '4.\tWhat city do you live in? Please include state if applicable. (i.e. New York, NY)',
       '5.\tWhat type of area do you live in? \t',
       '6.\tWhat age group do you fall into? ', '7.\tSex\t',
       '8.\tIf applicable, are you pregnant? ',
       '10.\tIf applicable, do you have periods/a menstrual cycle? ',
       '11.\tWhich of the following best describes your ancestry? Select all that apply. ',
       '12.\tIs your household income from all sources? ',
       '13.\tWhat is your highest educational level achieved? ',
       '14.\tAre you a healthcare professional? ',
       '15.\tWhat is your current work status? Select all that apply'],
      dtype='object')

In [26]:
demo_dict={"1.\tWhat is your height?":"height",
                    "2.\tWhat is your weight?":"weight",
                    "3.\tIn which country do you currently reside? ":"country",
                    "4.\tWhat city do you live in? Please include state if applicable. (i.e. New York, NY)":"city/state",
                    "5.\tWhat type of area do you live in? \t":"area",
                    '6.\tWhat age group do you fall into? ':"age_group",
                    '7.\tSex\t':"sex",
                    '8.\tIf applicable, are you pregnant? ':"pregnant",
                     '10.\tIf applicable, do you have periods/a menstrual cycle? ':"menstruation",
                     '11.\tWhich of the following best describes your ancestry? Select all that apply. ':"ancestry",
                     '12.\tIs your household income from all sources? ':"income_sources",
                     '13.\tWhat is your highest educational level achieved? ':"education_highest",
                     '14.\tAre you a healthcare professional? ':"healthcare_prof",
                    "15.\tWhat is your current work status? Select all that apply":"work_status"}


covid_dict={"16. Did you have COVID-19 infection? if no don't proceed":"previous_infection",
                    "What type of test was used to test you for COVID-19?":"test_type",
                    "What date were you tested for COVID-19? If you don't know the exact date, please choose your best approximation.":"date_tested",
                    "Did you consult with a physician(s) for your COVID-19 symptoms? ":"consultation_sought",
                    "Who did you consult for your COVID-19 symptoms? ":"consultant_type",
                    "When did your symptoms begin? ":"symptoms_start_time",
                    '2.\tAre you still experiencing symptoms? ':"symptoms_persistance",
                    '3.\tHow many days total did you experience symptoms? ':"days_with_symptoms",
                    '4.Lifestyle & Pre-existing Conditions:            1.\tDid you have any of these pre-existing conditions/diagnoses or did you experience any of the following pre-COVID-19?':"pre-existing conditions",
                     '6.\tDid any of your pre-existing conditions change during your COVID19 symptoms?':"change in pre-exsiting condition",
                     '7.\tIn the month before becoming ill if you were sick, or in the previous month if you were not, were you a regular, occasional, or never smoker?':"smoker",}



treat_dict={'1.\tWere you hospitalized?  ': "hospitalized",
                    "What type of test was used to test you for COVID-19?":"test_type",
                    "2.\tIf yes: how long were you hospitalized? [Number of days] ":"hospitlization_duration",
                    "3.\tDid you receive oxygen support in the hospital? ":"oxygen_support",
                    "Who did you consult for your COVID-19 symptoms? ":"consultant_type",
                    "Have you tried any of the following treatments for your COVID19 symptoms, if yes, how helpful it was? This includes Prescription or off-the-counter Medications, or Alternative Treatments.":"OTC_prescription_alternate_use",
                    'Have you got the Covid-19 vaccine?':"vaccine",
                      "How many shots have you got?":"vaccine_shots",
                      'When you get the vaccine':"vaccine_date",
                    'Have you got infected with COVID-19 after vaccination?':"infection_post_vaccination"}





In [29]:
codins=pd.DataFrame(columns=["column","rename"])
c=0
for d in [demo_dict,covid_dict,treat_dict,mos_dict,other_dict]:
    for k,v in d.items():
        codins.loc[c,"column"]=k
        codins.loc[c,"rename"]=v
        c=c+1
codins
codins.to_csv("column_names.csv")

In [8]:
## let us rename some columns for easier wrangling:
Demo.rename(columns={"1.\tWhat is your height?":"height",
                    "2.\tWhat is your weight?":"weight",
                    "3.\tIn which country do you currently reside? ":"country",
                    "4.\tWhat city do you live in? Please include state if applicable. (i.e. New York, NY)":"city/state",
                    "5.\tWhat type of area do you live in? \t":"area",
                    '6.\tWhat age group do you fall into? ':"age_group",
                    '7.\tSex\t':"sex",
                    '8.\tIf applicable, are you pregnant? ':"pregnant",
                     '10.\tIf applicable, do you have periods/a menstrual cycle? ':"menstruation",
                     '11.\tWhich of the following best describes your ancestry? Select all that apply. ':"ancestry",
                     '12.\tIs your household income from all sources? ':"income_sources",
                     '13.\tWhat is your highest educational level achieved? ':"education_highest",
                     '14.\tAre you a healthcare professional? ':"healthcare_prof",
                    "15.\tWhat is your current work status? Select all that apply":"work_status"},
            inplace=True)
Demo.columns

Index(['PatientID', 'height', 'weight', 'country', 'city/state', 'area',
       'age_group', 'sex', 'pregnant', 'menstruation', 'ancestry',
       'income_sources', 'education_highest', 'healthcare_prof',
       'work_status'],
      dtype='object')

In [9]:
full_report(Demo)

Missing Values
 PatientID              0
height                15
weight                16
country               15
city/state            55
area                  39
age_group             47
sex                   19
pregnant             732
menstruation         731
ancestry              87
income_sources        69
education_highest     39
healthcare_prof       57
work_status           32
dtype: int64 

Data types 
 PatientID            object
height               object
weight               object
country              object
city/state           object
area                 object
age_group            object
sex                  object
pregnant             object
menstruation         object
ancestry             object
income_sources       object
education_highest    object
healthcare_prof      object
work_status          object
dtype: object 

 Quick Description 
 <bound method NDFrame.describe of      PatientID            height  weight             country       city/state  \
0        

* **First observation there are alot of missing values** 
* **Second observation there are columns of wrong data type**
* **Third observation there are alot of units to deal with**

### 1.1 Dealing with units

```python 
## let us correct some of the data types
Demo["height"]=Demo["height"].astype(float)
Demo["weight"]=Demo["weight"].astype(float)

```


This code returns a value error

ooooh we have a problem with units in weight and height ..

In [10]:
## let us start with height 
for value in Demo.height:
    print(value)

161
5 feet 2 inches 
5 3' 
5' 3''
5.2 feet
5'7
5'7
5'5"
5’4
5 ft 8 inch
174
5 feet
5 feet
5 ft
5'2 
5.4 feet
5’7 
155
169
130 cm
163
5”3’
173
5 10”
153
170
187
5'10
180
6 ft
176cm
170 cm
5”2
160 cm
5’ 3”
157cm
178cm
169
165
162
5’3
5 ft 8 inch
5'2
5.1
159cm
185
163
160cm
176
165
161 cm
178
165
5 feet 8 inches
nan
169 cm 
157 cm 
175
160
159
163cm 
175 cm
168
172 c.m
178
172 cm 
165
170
165
164
180
160
165
155 Cm 
169
160cm
174
165
156
165
162
157
159
165
172 cm
158 cm 
155
165
70
167
157
159
160
165
180 cm
165
160
180
170
150 cm
165
163
171
155
174
168 cm
174 cm
174 cm
181
172
155
155 Cm
175
None
175
171
185 cm 
150cm
171cm
166 cm
149
185
167
163
176cm
168 cm
160 cm
174 cm 
180 cm 
155
5'5
156
185cm
180
175
151 cm
183 cm 
150 cm 
155 cm
158 Cm
157cm
186
140
5ft 11 inches
178
168
165
nan
145
165cm
160cm
157cm
158
1.63 meter
5feet 3 inches
150cm
156 cm
5 ft 8 inch 
183 cm
5’9
5'6"
5 11
5 feet 3 inches
5'8
175 cm
160
5 feet 6 inches 
5'6
5 ft 3 inch
5’10
5’5
166
175 Cm
177.8cm
F ft 3 inch

173
170cm
150
154 cm
166
180
172
183
168
153
165
161
156
163
165
165
150
164
163 cm
162
165
178 cm
5 feet
175 cm
158
6'2"
5'4"
150 cm
165
150 cm
162
5'10"
165 cm
5'5"
159
164
163
5'7"
163
160cm
172 cm
155cm
162
5 ft
173 cm
145cm
164
162
159
164
5ft 3in
163
160
170
170
158
155
171
180
169
180
170
180
160
169
183
175 cm
173 cm
167
182cm
190 cm
178cm
178 cm
170 cm 
165 cm
179cm
186
183
181
189
170
166
181 cm
180 cm
174
163
157
184
159
176
178
169
162
174
160
170
157
174
163
155
172
176 cm
158
173 cm
181 cm
181 cm
180 cm
166 cm
174 cm
174 cm
163
176
180 cm
178
177
170
171 cm
172 cm
175 cm
180
140 cm
180 cm
182 cm
162 cm
173 cm
176 cm
163
170 cm 
179 cm
1.7
1.78
1.65
1.72
1.64


In [11]:
## ok now that I skimmed all the values and I have a plan 
inch_ft=["'","feet","inch","ft","’",'"',"foot","”"]
nulls=["none","nan","null"]

inch_unit=[]
cm_unit=[]
meter_unit=[]
null_value=[]
other_prob=[]
no_prob=[]
for e,value in enumerate(Demo["height"]):
    if ("cm" in str(value).lower() )| ("c.m" in str(value).lower()):
        cm_unit.append(e)
    elif any(elem in str(value).lower() for elem in inch_ft):
        inch_unit.append(e)
    elif ("meter" in str(value).lower()):
        meter_unit.append(e)
    elif (any(elem in str(value).lower() for elem in nulls)) or str(value)=="0":
        null_value.append(e)
    else: 
        try:
            float(value)
            no_prob.append(e)
        except:
            other_prob.append(e)


In [12]:
Demo["height"][inch_unit]

1       5 feet 2 inches 
2                  5 3' 
3                 5' 3''
4               5.2 feet
5                    5'7
              ...       
2468               5'10"
2470                5'5"
2474                5'7"
2480                5 ft
2487             5ft 3in
Name: height, Length: 483, dtype: object

In [13]:
Demo["height"][cm_unit]

19       130 cm
30        176cm
31       170 cm
33       160 cm
35        157cm
         ...   
2560     162 cm
2561     173 cm
2562     176 cm
2564    170 cm 
2565     179 cm
Name: height, Length: 634, dtype: object

In [14]:
Demo["height"][null_value]

54       NaN
113     None
147      NaN
278     None
336     None
349      NaN
350     None
722      NaN
941      NaN
1228     NaN
1229     NaN
1272    None
1656     NaN
1900     NaN
2008     NaN
Name: height, dtype: object

In [15]:
Demo["height"][meter_unit]

153    1.63 meter
195    5.5 meters
328    1 meter 73
Name: height, dtype: object

In [16]:
Demo["height"][other_prob]

161       5 11
209       1.8m
223       1.6m
228       1.8m
232      1.85m
234      1.73m
236      1.95m
237      1.60m
238      1.90m
239       1.7m
240      1.80m
241      1.75m
242      1.62m
243      1.65m
244      1.75m
246       1.7m
247       1.7m
248       1.8m
249       1.8m
250       1.6m
260       1.8m
319       1,61
327       1,75
337       1m82
375       1m92
403        5 2
419          F
421      1.66m
422       1,70
423     1.68 m
425      1.67m
426      1.75m
427      1.77m
428      1.90m
429      1.77m
430      1.77m
431      1.77m
432      1.68m
512        5 9
587        5 7
594        5,2
596        5 5
659      1.68m
1218       5,5
1403      5,10
1561      5 10
1581         ⅝
1584      5.6m
1778      1m66
1783      88kh
1801      1m73
1802      1m85
1803      1m94
1809      1m56
1824      1m66
2209     187cn
Name: height, dtype: object

In [17]:
Demo["height"][no_prob]

0        161
10       174
17       155
18       169
20       163
        ... 
2566     1.7
2567    1.78
2568    1.65
2569    1.72
2570    1.64
Name: height, Length: 1380, dtype: object

In [18]:
##we will work all in cm 
import re
for e,value in enumerate(Demo.height.values):
    if e in cm_unit:
        Demo.loc[e,"height"]=re.sub("\D","",value)
Demo["height"][cm_unit]

19      130
30      176
31      170
33      160
35      157
       ... 
2560    162
2561    173
2562    176
2564    170
2565    179
Name: height, Length: 634, dtype: object

In [19]:
#for the inches data , we will have to play around a bit 
# we will create 2-item lists for each value 
# multiply the first value with 12 and the second with 2.54


#remove all non-digit characters using regex 
for e,value in enumerate(Demo.height.values):
    if e in inch_unit:
        temp=re.findall("[0-9]+", value)
        if len(temp)<2:
            temp.append(0)
        in_cm=int(temp[0])*12+int(temp[1])*2.54
        Demo.loc[e,"height"]=in_cm
Demo["height"][inch_unit]

1       65.08
2       67.62
3       67.62
4       65.08
5       77.78
        ...  
2468     85.4
2470     72.7
2474    77.78
2480       60
2487    67.62
Name: height, Length: 483, dtype: object

In [20]:
# replace all the null values with np.nan
for e,value in enumerate(Demo.height.values):
    if e in null_value:
        Demo.loc[e,"height"]=np.nan

In [21]:
## finally we have meters as a problem , we have meters saved in the meters list and others wrongly classified
#as other problems  and as not a problem ,let us deal with the one at a time 
for e,value in enumerate(Demo.height.values):
    if e in meter_unit:
        in_cm=re.sub("\D","",value)
        Demo.loc[e,"height"]=float(in_cm)*100
        

In [22]:
### for no_prob one is simple 
## if a number is greater than 3 but less than or equal 6 it is probably in feet-inch 
## if it is less than 3 it is probably in meters
## else it is in cm 
for e,value in enumerate(Demo.height.values):
    if e in no_prob:
        if float(value)<3:
            final=float(value)*100
        elif float(value) < 7:
            temp=re.findall("[0-9]+", str(value))
            if len(temp)<2:
                temp.append(0)
            final=int(temp[0])*12+int(temp[1])*2.54
        else:
            final=float(value)
        Demo.loc[e,"height"]=final


In [23]:
## last but not least the other_prob values
# let us first take a close look at it 
for e,i in enumerate(Demo.height[other_prob]):
    print(i,other_prob[e])

5 11 161
1.8m 209
1.6m 223
1.8m 228
1.85m 232
1.73m 234
1.95m 236
1.60m 237
1.90m 238
1.7m 239
1.80m 240
1.75m 241
1.62m 242
1.65m 243
1.75m 244
1.7m 246
1.7m 247
1.8m 248
1.8m 249
1.6m 250
1.8m 260
1,61 319
1,75 327
1m82 337
1m92 375
5 2 403
F 419
1.66m 421
1,70 422
1.68 m 423
1.67m 425
1.75m 426
1.77m 427
1.90m 428
1.77m 429
1.77m 430
1.77m 431
1.68m 432
5 9 512
5 7 587
5,2 594
5 5 596
1.68m 659
5,5 1218
5,10 1403
5 10 1561
⅝ 1581
5.6m 1584
1m66 1778
88kh 1783
1m73 1801
1m85 1802
1m94 1803
1m56 1809
1m66 1824
187cn 2209


In [24]:
## gotta crack my fingers for this one
### before creating a for loop there are three bizarre values that i shall correct manually 
Demo.loc[1581,"height"]=5*12+8*2.54
other_prob.remove(1581)
Demo.loc[1783,"height"]=np.nan
other_prob.remove(1783)
Demo.loc[419,"height"]=np.nan
other_prob.remove(419)


for e,value in enumerate(Demo.height.values):
    if e in other_prob:
        if "m" in str(value):
            in_cm=re.sub("\D","",value)
            Demo.loc[e,"height"]=float(in_cm)*100
        elif "c" in str(value):
            in_cm=re.sub("\D","",value)
            Demo.loc[e,"height"]=float(in_cm)
        else:
            temp=re.findall("[0-9]+", str(value))
            if len(temp)<2:
                temp.append(0)
            final=int(temp[0])*12+int(temp[1])*2.54
            Demo.loc[e,"height"]=final
            
            
            
            
            
            

In [25]:
Demo.height=Demo.height.astype(float)

In [26]:
### now let us look into weights before we call it a day 
### ok weight is simple I skimmed through the values just simple for loop will suffice
for e,value in enumerate(Demo.weight):
    if "k" in str(value):
        final=re.sub("\D","",value)
        Demo.loc[e,"weight"]=float(final)
    elif "pound" in str(value) or "lb" in str(value):
        final=re.sub("\D","",value)
        Demo.loc[e,"weight"]=float(final)*0.45
    else:
        try:
            Demo.loc[e,"weight"]=float(value)
        except:
            Demo.loc[e,"weight"]=np.nan
Demo.weight=Demo.weight.astype(float)
            
        
        
        
    

In [27]:
## now that we have dealth with height and weight 
## we will move on onto the rest of the Demo df 
full_report(Demo)

Missing Values
 PatientID              0
height                17
weight                58
country               15
city/state            55
area                  39
age_group             47
sex                   19
pregnant             732
menstruation         731
ancestry              87
income_sources        69
education_highest     39
healthcare_prof       57
work_status           32
dtype: int64 

Data types 
 PatientID             object
height               float64
weight               float64
country               object
city/state            object
area                  object
age_group             object
sex                   object
pregnant              object
menstruation          object
ancestry              object
income_sources        object
education_highest     object
healthcare_prof       object
work_status           object
dtype: object 

 Quick Description 
 <bound method NDFrame.describe of      PatientID  height  weight             country       city/state  \
0   

### 1.2 Dealing with Cities and Countries

In [28]:
##ok for the country column we seem to have some problems let us skimm throught the values then start wrangling 
for i in Demo.country.value_counts().items():
    print(i)


('Egypt', 446)
('India', 303)
('Pakistan', 274)
('Syria', 197)
('Egypt ', 180)
('Pakistan ', 154)
('Syria ', 99)
('India ', 90)
('Afghanistan ', 84)
('Iran', 66)
('egypt', 62)
('Afghanistan', 60)
('Algeria', 43)
('Palestine', 39)
('UAE', 34)
('Yemen ', 32)
('Libya', 28)
('syria', 24)
('Yemen', 24)
('Qatar', 20)
('iran', 19)
('Saudi Arabia ', 16)
('Libya ', 13)
('afghanistan', 8)
('Saudi Arabia', 8)
('Germany', 7)
('IRAN', 7)
('USA', 7)
('Qena ', 6)
('Sudan', 6)
('Cairo', 6)
('EGYPT', 5)
('Turkey', 5)
('تركيا', 4)
('Algeria ', 4)
('hama', 4)
('egypt ', 4)
('Damascus', 4)
('pakistan', 3)
('KSA', 3)
('United States', 3)
('Lebanon', 3)
('libya', 3)
('Qena', 3)
('INDIA', 3)
('Afghanisatn', 2)
('Tobruk,libya ', 2)
('qatar', 2)
('United Kingdom ', 2)
('USA ', 2)
('Germany ', 2)
('United Arab Emirates ', 2)
('assuit', 2)
('Deutschland', 2)
('Libya tobruk', 2)
('Qatar ', 2)
('مصر', 2)
('Aleppo', 2)
('Saudia arabia', 2)
('Italy', 2)
('Lebanon ', 2)
('Australia', 2)
('Libya/Tobruk ', 2)
('Alexand

problems with country column:
* caps vs lower 
* some wrote cities as well
* some spelling mistakes 
* unnecessary spaces in country name 
* some dashes and slashes 

In [29]:
## let's do some cleaning 

#caps , spaces and punctuations problem
##  and spelling mistakes problem
from autocorrect import Speller

spell = Speller(lang='en')


Demo.country=Demo.country.apply(lambda x :spell(re.sub(r'[^\w\s]'," ",str(x).lower().strip())))

# extra cities problem:
# and egypt cities  and arabic problem 
egypt=["qena","cairo","gize","giza","assyut","assuit","almenofia","tanta","مصر","alshohadaa","assyt",
      "qalubia"]

countries=['egypt', 'pakistan', 'india', 'syria', 'afghanistan', 'iran','yemen','algeria','libya','palestine','uae','saudi arabia','qatar','usa']
for e,text in enumerate(Demo.country):
    for country in countries:
        if country in text:
            Demo.loc[e,"country"]=country
    for city in egypt:
        if city in text:
            Demo.loc[e,"country"]="egypt"

In [30]:
for i in Demo.country.value_counts().items():
    print(i)

('egypt', 719)
('pakistan', 436)
('india', 399)
('syria', 330)
('afghanistan', 157)
('iran', 92)
('yemen', 56)
('libya', 56)
('algeria', 48)
('palestine', 40)
('uae', 35)
('saudi arabia', 31)
('qatar', 24)
('usa', 11)
('nan', 10)
('germany', 9)
('ena', 9)
('sudan', 7)
('lebanon', 5)
('none', 5)
('assist', 5)
('turkey', 5)
('damascus', 5)
('ham', 4)
('تركيا', 4)
('united states', 3)
('united arab emirates', 3)
('sa', 3)
('alexandria', 3)
('bengali', 2)
('italy', 2)
('deutschland', 2)
('oman', 2)
('united states of america', 2)
('australia', 2)
('actor', 2)
('united kingdom', 2)
('aleppo', 2)
('ashton', 2)
('denmark', 2)
('gia', 2)
('uk', 1)
('saudi', 1)
('russia', 1)
('79', 1)
('went', 1)
('tunisia', 1)
('deutsch', 1)
('fujairah', 1)
('t', 1)
('united arab of emirates', 1)
('nepal', 1)
('jordan', 1)
('digit', 1)
('turkey  patient 1 ', 1)
('turkey ', 1)
('passion', 1)
('istanbul', 1)
('bangalore  karnataka', 1)
('england', 1)
('sabha medical center', 1)
('80', 1)
('africa', 1)
('gb', 1)


In [31]:
### the rest of the cleaning is very specific for code I will save this dataframe and fix it by hand 
##Demo.to_excel("demo.xlsx",engine="openpyxl")

In [28]:
Demo2=pd.read_excel("demo.xlsx",engine="openpyxl")


In [29]:
for i in Demo2.country.value_counts().items():
    print(i)

('egypt', 744)
('pakistan', 438)
('india', 403)
('syria', 340)
('afghanistan', 157)
('iran', 92)
('libya', 60)
('yemen', 57)
('algeria', 48)
('palestine', 41)
('uae', 40)
('ksa', 33)
('qatar', 24)
('usa', 18)
('turkey', 14)
('germany', 12)
('sudan', 7)
('lebanon', 5)
('uk', 4)
('oman', 2)
('italy', 2)
('australia', 2)
('denmark', 2)
('kuwait', 1)
('jordan', 1)
('bahrain', 1)
('scotland', 1)
('tunisia', 1)
('russia', 1)
('canada', 1)


In [30]:
## one last thing 
Demo2.country=Demo.country=Demo2.country.apply(lambda x : str(x).lower().strip())
for e,i in enumerate(Demo2.country):
    if i =="none" or i=="nan":
        Demo2.loc[e,"country"]=np.nan

In [31]:
for i in Demo2.country.value_counts().items():
    print(i)

('egypt', 744)
('pakistan', 438)
('india', 403)
('syria', 340)
('afghanistan', 157)
('iran', 92)
('libya', 60)
('yemen', 57)
('algeria', 48)
('palestine', 41)
('uae', 40)
('ksa', 33)
('qatar', 24)
('usa', 18)
('turkey', 14)
('germany', 12)
('sudan', 7)
('lebanon', 5)
('uk', 4)
('oman', 2)
('italy', 2)
('australia', 2)
('denmark', 2)
('kuwait', 1)
('jordan', 1)
('bahrain', 1)
('scotland', 1)
('tunisia', 1)
('russia', 1)
('canada', 1)


In [32]:
Demo2.head()

,PatientID,height,weight,country,city/state,area,age_group,sex,pregnant,menstruation,ancestry,income_sources,education_highest,healthcare_prof
0,0,161.00,57.0,egypt,cairo, Urban, 18-29, Female, No, Yes," Middle Eastern, North African (Lebanese, Ira..."," Less than $10,000", Standard college degree,No
1,1,65.08,70.0,pakistan,punjab, Urban, 18-29, Female, No," No, other reason"," Asian, South Asian, southeast Asian (Chinese..."," Less than $10,000", Partial college (at least one year) or speci...,Yes
2,2,67.62,61.0,pakistan,punjab, Suburban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," $10,000 to less than $20,000", Graduate professional degree including maste...,Yes
3,3,67.62,64.0,pakistan,punjab, Urban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," Less than $10,000", High school degree or GED,Yes
4,4,65.08,54.0,pakistan,punjab, Urban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," $10,000 to less than $20,000", Graduate professional degree including maste...,Yes


In [6]:
### now with the city column 
## probably the last column that will require much effort 
### let us skim through all the values 

### lesson learned earlier: remove unncessary stuff first 
#Demo2["city/state"]=Demo2["city/state"].apply(lambda x :re.sub(r'[^\w\s]'," ",str(x).lower().strip()))


##for every country let us see the city variations 
## we will create a function cuz we will need it alot 
def city_report():
    for country in set(Demo.country):
        temp=Demo[Demo.country==country]
        print(f"----------\n{country}\n")
        for i in temp["city/state"].value_counts().items():
            print(i)
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
scotland

('glasgow  scotland', 1)
----------
iran

('tehran', 36)
('sharjah', 21)
('karaj', 11)
('azerbaijan', 10)
('yazd', 4)
('isfahan', 4)
('sari', 2)
('saveh', 1)
('tabriz', 1)
('khorramabad', 1)
('qazvin', 1)
----------
pakistan

('punjab', 268)
('islamabad', 72)
('sindh', 70)
('pakistan', 5)
('bhakkar', 3)
('gilgit baltistan', 2)
('sharjah', 1)
('gujrat', 1)
----------
italy

----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
usa

('houston', 3)
('lubbock  tx', 3)
('new york', 2)
('wheaton  il', 1)
('lubbock', 1)
('miami   florida', 1)
('woodbridge  virginia', 1)
('dammam', 1)
('kansas', 1)
('dearborn  mi', 1)
('va', 1)
('california', 1)
----------
kuwait

('kuwait', 1)
----------
jordan

('irbid', 1)
----------
ksa

('riyadh', 13)
('jeddah', 6)
('madina', 5)
('mekkah', 4)
('jizan', 3)
('dammam', 2)
------

In [34]:
## let us finish one country at a time
## for ksa (lebanon and bahrain are ok)

for e,value in enumerate(Demo2["city/state"]):
    if "makkah" in value:
        Demo2.loc[e,"city/state"]="mekkah"
    elif  "jeddah" in value:
        Demo2.loc[e,"city/state"]="jeddah"
    elif ("riyadh" in value) |("read" in value) :
        Demo2.loc[e,"city/state"]="riyadh"
    elif ("المدينة المنورة" in value)  :
        Demo2.loc[e,"city/state"]="madina"
    elif ("الدمام" in value) |("read" in value) :
        Demo2.loc[e,"city/state"]="dammam"


    

In [35]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
('nan', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
('nan', 1)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('nan', 12)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('nan', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('houston', 3)
('lubbock  tx', 3)
('new york', 2)
('woodbridge  virginia', 1)
('miami   florida', 1)
('kansas', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('nan', 1)
('dearborn  mi', 1)
('dammam', 1)
('v

In [36]:
## for india after skimming the values and doing some research, I will get only states from the data instead of
## cities 

for e,value in enumerate(Demo2["city/state"]):
    if Demo2.loc[e,"country"]=="india":
        try:
            state_list=value.split(" ")
            if ((state_list[-1]=="india") | (state_list[-1]=='') |(len(state_list[-1])<3) | (state_list[-1] =="state") | (state_list[-1]=="urban")):
                Demo2.loc[e,"city/state"]=state_list[0]
            else:
                Demo2.loc[e,"city/state"]=state_list[-1]
        except:
            pass
    else:
        pass

In [37]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
('nan', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
('nan', 1)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('nan', 12)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('nan', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('houston', 3)
('lubbock  tx', 3)
('new york', 2)
('woodbridge  virginia', 1)
('miami   florida', 1)
('kansas', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('nan', 1)
('dearborn  mi', 1)
('dammam', 1)
('v

In [38]:
### final adjustments for india cities/states
for e,value in enumerate(Demo2["city/state"]):
    if ("india" in value) | ("nan" in value) | (len(value)<2)|("state" in value):
        Demo2.loc[e,"city/state"]=np.nan
    elif  "bengaluru" in value:
        Demo2.loc[e,"city/state"]="bangalore"
    elif  ("karnaraka" in value) | ("karna" in value):
        Demo2.loc[e,"city/state"]="karnataka"


In [39]:
###russia is ok 
### now for uae and sudan

for e,value in enumerate(Demo2["city/state"]):
    if "khatoum" in str(value):
        Demo2.loc[e,"city/state"]="khartoum"
    elif "sh" in str(value):
        Demo2.loc[e,"city/state"]="sharjah"
    elif  "fuj" in str(value):
        Demo2.loc[e,"city/state"]="fujairah"
    elif ("uae" in str(value)) :
        Demo2.loc[e,"city/state"]=np.nan

In [40]:
### for afghanistan and yemen 
for e,value in enumerate(Demo2["city/state"]):
    if "parvan" in str(value):
        Demo2.loc[e,"city/state"]="parwan"
    elif "paktya" in str(value):
        Demo2.loc[e,"city/state"]="paktika"
    elif  "kabul" in str(value):
        Demo2.loc[e,"city/state"]="kabul"
    elif ("afghanistan" in str(value)) :
        Demo2.loc[e,"city/state"]=np.nan
    else:
        ### mukalla is a region , in which hadhramout city exists 
        if "out side mukalla" in str(value) :
            Demo2.loc[e,"city/state"]=np.nan
        elif ("muk" in str(value)) | ("had" in str(value)):
            Demo2.loc[e,"city/state"]="mukalla"
 

In [41]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('lubbock  tx', 3)
('houston', 3)
('new york', 2)
('va', 1)
('woodbridge  virginia', 1)
('miami   florida', 1)
('dearborn  mi', 1)
('dammam', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('kansas', 1)
----------
russia

('ufa', 1)
----------
uk

('sh

In [42]:
## for egypt ....
# we group all the villages, towns and cities into governates instead

##defining cities and towns of governates 
beheira=["beheira","albuheira","abohommos","damanhour",'kafr eldawar',"behira","edkou","kom hamada","damanhur",
        "behera","kafreldawar","كفرالدوار","كوم حماده","البحيره", "البحيرة","damnhour","eitay","edku",'behaira',"abohommous","albuhira"]

gharbia=["tanta","samanoud","الغربيه","gharbia","kafr el zayat","mahallah","elzayat","elmahalla","samanud","zeifta","mansoira"]

dakahlia= ["masnoura","mansoura","mit","dakahlia","mit","manzalla","الدقهليه","mansura"]

sharqia =["zagazig","menia el qamh","el salhia","kafr sacr","menia elkamh","منيا القمح","minia alkameh","fakous","minia el quamh"]

minya=["minya",'menia',"minia","menya"]

cairo=["nasr","cario","cairo","catio","naser"]
fayoum=["fayoum"]
beni_suef=["beni suef"]

alexandria=["alexandria","alexandrine","اسكندريه","alexandra","alexandira","alexandia"]
munfia=["elsadat","tala","menofeya","منوفيه","menofia"]

giza=["6 of october   giza","el saaf geza egypt","6th october","elsaff","october"]

qalubia=["qaliobia","obour","banha","benha","qal","khankah"]

qena=["qena"]

assuit=["assiut","assuit"]


governates={"beheira":beheira,"gharbia":gharbia,"dakahlia":dakahlia,"sharqia":sharqia,"minya":minya
            ,"cairo":cairo,"alexandria":alexandria,"munfia":munfia,"giza":giza,
            "qalubia":qalubia,"qena":qena,"fayoum":fayoum,"beni suef":beni_suef,"assuit":assuit}







In [43]:
from tqdm import tqdm 
for gov,cities in tqdm(governates.items()):
    for e,value in enumerate(Demo2["city/state"]):
        if any(elem in str(value).lower() for elem in cities):
            Demo2.loc[e,"city/state"]=gov
for e,value in enumerate(Demo2["city/state"]):
    if (not any(elem in str(value).lower() for elem in list(governates.keys()))) & (Demo2.loc[e,"country"]=="egypt") :
        Demo2.loc[e,"city/state"]="other"
        

            

100%|███████████████████████████████████████████| 14/14 [00:00<00:00, 43.47it/s]


In [44]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('lubbock  tx', 3)
('houston', 3)
('new york', 2)
('va', 1)
('woodbridge  virginia', 1)
('miami   florida', 1)
('dearborn  mi', 1)
('dammam', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('kansas', 1)
----------
russia

('ufa', 1)
----------
uk

('sh

In [45]:
##libya ,italy,australia,iran and turkey

for e,value in enumerate(Demo2["city/state"]):
    if "موغلا" in str(value):
        Demo2.loc[e,"city/state"]="mugla"
    elif "كربوك" in str(value):
        Demo2.loc[e,"city/state"]="kurbouk"
    elif  "milan" in str(value):
        Demo2.loc[e,"city/state"]="milano"
    elif (("tobruk" in str(value)) |("toubrok" in str(value))| ("tourok" in str(value))) :
        Demo2.loc[e,"city/state"]="tobruk"
    elif  "sabha" in str(value):
        Demo2.loc[e,"city/state"]="sabha"
    elif  "melborne" in str(value):
        Demo2.loc[e,"city/state"]="melbourne"

In [46]:
##syria 
for e,value in enumerate(Demo2["city/state"]):
    if "damascus" in str(value):
        Demo2.loc[e,"city/state"]="damascus"
    elif "إدلب" in str(value):
        Demo2.loc[e,"city/state"]="idlib"
    elif  "milano" in str(value):
        Demo2.loc[e,"city/state"]="milano"
    elif (("حلب" in str(value)) |("aleppo" in str(value))| ("alepp" in str(value))) :
        Demo2.loc[e,"city/state"]="tobruk"
    elif  "hama" in str(value):
        Demo2.loc[e,"city/state"]="hama"
    elif  "idleb" in str(value):
        Demo2.loc[e,"city/state"]="idlib"
    elif  "syria" in str(value):
        Demo2.loc[e,"city/state"]=np.nan
    elif  "azerbaijan" in str(value):
        Demo2.loc[e,"city/state"]="azerbaijan"

In [47]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('lubbock  tx', 3)
('houston', 3)
('new york', 2)
('va', 1)
('woodbridge  virginia', 1)
('miami   florida', 1)
('dearborn  mi', 1)
('dammam', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('kansas', 1)
----------
russia

('ufa', 1)
----------
uk

('sh

In [48]:
###finally pakistan
for e,value in enumerate(Demo2["city/state"]):
    if "punjab" in str(value):
        Demo2.loc[e,"city/state"]="punjab"
    elif "lahore" in str(value):
        Demo2.loc[e,"city/state"]="lahore"
    elif  "islamabad" in str(value):
        Demo2.loc[e,"city/state"]="islamabad"
    elif "rawalpindi" in str(value):
        Demo2.loc[e,"city/state"]="rawalpindi"
    elif  "gujranwala" in str(value):
        Demo2.loc[e,"city/state"]="gujranwala"




In [49]:
punjab=["lahore","maltan","multan","lodhran",'jhang',"rawalpindi","gujranwala","sialkot","bhakkar"
       "islamabad","hafizabad","faisalabad","mianwali","lahoڑe","muzaffargarh","rahimyarkhan",
       "gago","daska","sargodha","rahimyarkhan","narowal","bure","chakwal"]

sindh=["karachi","ghotki","larkana"]


nulls=["nill","gb","no","wah"]
for e,value in enumerate(Demo2["city/state"]):
    if any(elem in str(value).lower() for elem in punjab):
                Demo2.loc[e,"city/state"]="punjab"
    elif  any(elem in str(value).lower() for elem in sindh):
                  Demo2.loc[e,"city/state"]="sindh"
    elif  any(elem in str(value).lower() for elem in nulls):
                  Demo2.loc[e,"city/state"]=np.nan


                


In [50]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
jordan

('irbid', 1)
----------
algeria

('algiers', 35)
('tlemcen', 6)
('blida', 3)
('tebessa', 2)
('tlemcen algeria', 1)
('tizi ouizoi', 1)
----------
bahrain

('manama', 1)
----------
libya

('tobruk', 49)
('sabha', 9)
('benghazi', 1)
----------
yemen

('mukalla', 52)
('sana a', 2)
('sharjah', 2)
----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
germany

('munich', 2)
('wittenberge', 1)
('bavaria', 1)
('zaarland', 1)
('sharjah', 1)
('bonn', 1)
('berlin', 1)
('staßfurt', 1)
('wettinberge', 1)
('munech', 1)
----------
kuwait

('kuwait', 1)
----------
usa

('lubbock  tx', 3)
('houston', 3)
('new york', 2)
('va', 1)
('woodbridge  virginia', 1)
('miami   florida', 1)
('dearborn  mi', 1)
('dammam', 1)
('lubbock', 1)
('california', 1)
('wheaton  il', 1)
('kansas', 1)
----------
russia

('ufa', 1)
----------
uk

('sh

In [52]:
##finally let us save the demo df
#Demo2.drop("Unnamed: 0",inplace=True,axis=1)
Demo2["work_status"]=Demo["work_status"]
Demo2.to_excel("demo.xlsx",index=False)

In [53]:
Demo2.head()

,PatientID,height,weight,country,city/state,area,age_group,sex,pregnant,menstruation,ancestry,income_sources,education_highest,healthcare_prof,work_status
0,0,161.00,57.0,egypt,cairo, Urban, 18-29, Female, No, Yes," Middle Eastern, North African (Lebanese, Ira..."," Less than $10,000", Standard college degree,No, Working full-time
1,1,65.08,70.0,pakistan,punjab, Urban, 18-29, Female, No," No, other reason"," Asian, South Asian, southeast Asian (Chinese..."," Less than $10,000", Partial college (at least one year) or speci...,Yes, Student
2,2,67.62,61.0,pakistan,punjab, Suburban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," $10,000 to less than $20,000", Graduate professional degree including maste...,Yes, Student
3,3,67.62,64.0,pakistan,punjab, Urban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," Less than $10,000", High school degree or GED,Yes, Student
4,4,65.08,54.0,pakistan,punjab, Urban, 18-29, Female, No, Yes," Asian, South Asian, southeast Asian (Chinese..."," $10,000 to less than $20,000", Graduate professional degree including maste...,Yes, Working full-time


In [3]:
###revising the india cities data and group them to states 
Demo=pd.read_excel("demo.xlsx",engine="openpyxl")

In [7]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
scotland

('glasgow  scotland', 1)
----------
iran

('tehran', 36)
('sharjah', 21)
('karaj', 11)
('azerbaijan', 10)
('yazd', 4)
('isfahan', 4)
('sari', 2)
('saveh', 1)
('tabriz', 1)
('khorramabad', 1)
('qazvin', 1)
----------
pakistan

('punjab', 268)
('islamabad', 72)
('sindh', 70)
('pakistan', 5)
('bhakkar', 3)
('gilgit baltistan', 2)
('sharjah', 1)
('gujrat', 1)
----------
italy

----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
usa

('houston', 3)
('lubbock  tx', 3)
('new york', 2)
('wheaton  il', 1)
('lubbock', 1)
('miami   florida', 1)
('woodbridge  virginia', 1)
('dammam', 1)
('kansas', 1)
('dearborn  mi', 1)
('va', 1)
('california', 1)
----------
kuwait

('kuwait', 1)
----------
jordan

('irbid', 1)
----------
ksa

('riyadh', 13)
('jeddah', 6)
('madina', 5)
('mekkah', 4)
('jizan', 3)
('dammam', 2)
------

In [10]:
Telangana=["hyderabad","karimnagar","adilabad"]
pradesh=["visakhapatnam","vijayawada","srikakulam","narsaraopet",
        "vizag","sompeta","guntur"]
Karnataka=["bangalore","mysore","raichur"]
gujarat=["ahmedabad"]
punjab=["patiala","bhatinda","gharat","chandigarh"]
chhattisgarh=["bilaspur"]
Maharashtra=["mumbai","pune","nagpur"]
Haryana=["gurgaon"]
other=["gilgit"]


states={"Telangana" :Telangana,
       "pradesh" :pradesh,
       "Karnataka" :Karnataka,
       "gujarat" :gujarat,
       "punjab" :punjab,
       "chhattisgarh" :chhattisgarh,
       "Maharashtra" : Maharashtra,
       "Haryana" :Haryana,
       "other":other}


from tqdm import tqdm 
for state,cities in tqdm(states.items()):
    for e,value in enumerate(Demo["city/state"]):
        if any(elem in str(value).lower() for elem in cities):
            Demo.loc[e,"city/state"]=state



100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 23.46it/s]


In [11]:
city_report()

----------
nan

----------
australia

('melbourne', 2)
----------
scotland

('glasgow  scotland', 1)
----------
iran

('tehran', 36)
('sharjah', 21)
('karaj', 11)
('azerbaijan', 10)
('yazd', 4)
('isfahan', 4)
('sari', 2)
('saveh', 1)
('tabriz', 1)
('khorramabad', 1)
('qazvin', 1)
----------
pakistan

('punjab', 268)
('islamabad', 72)
('sindh', 70)
('pakistan', 5)
('bhakkar', 3)
('other', 2)
('sharjah', 1)
('gujrat', 1)
----------
italy

----------
syria

('tobruk', 140)
('hama', 69)
('damascus', 46)
('latakia', 42)
('idlib', 11)
('homs', 10)
('tartous', 7)
('sharjah', 2)
('ariha', 1)
----------
usa

('houston', 3)
('lubbock  tx', 3)
('new york', 2)
('wheaton  il', 1)
('lubbock', 1)
('miami   florida', 1)
('woodbridge  virginia', 1)
('dammam', 1)
('kansas', 1)
('dearborn  mi', 1)
('va', 1)
('california', 1)
----------
kuwait

('kuwait', 1)
----------
jordan

('irbid', 1)
----------
ksa

('riyadh', 13)
('jeddah', 6)
('madina', 5)
('mekkah', 4)
('jizan', 3)
('dammam', 2)
----------
tunisi

In [12]:
Demo.to_excel("demo.xlsx",index=False)

## 2 Covid-related questions 

In [172]:
#Covid=pd.read_excel("covid_related.xlsx",engine="openpyxl")

In [171]:
Covid.head().T

,0,1,2,3,4
PatientID,0,1,2,3,4
previous_infection,Yes,Yes,Yes,Yes,Yes
test_type,Clinical symptoms,Nasal swab (PCR test),Nasal swab (PCR test),Nasal swab (PCR test),Nasal swab (PCR test)
date_tested,2021-01-14 00:00:00,NaT,NaT,2021-08-23 00:00:00,2021-08-16 00:00:00
consultation_sought,Yes,No,No,No,Yes
consultant_type,My primary care doctor/General practitioner,Alternative Medicine doctor,I have not seen any physician,I have not seen any physician,Alternative Medicine doctor
symptoms_start_time,2021-01-07 00:00:00,NaT,2021-10-27 00:00:00,2021-08-22 00:00:00,2021-08-15 00:00:00
symptoms_persistance,No,No,No,No,Yes
days_with_symptoms,NaN,7.0,NaN,NaN,10.0
pre-existing conditions, Nightmares," Environmental Allergies (dust, mold),  Chem..."," Insomnia,  Migraine", Anemia," Environmental Allergies (dust, mold),  Luci..."


In [306]:
full_report(Covid)

Missing Values
 PatientID                             0
previous_infection                   28
test_type                           119
date_tested                         386
consultation_sought                 131
consultant_type                     311
symptoms_start_time                 355
symptoms_persistance                146
days_with_symptoms                  241
pre-existing conditions             305
change in pre-exsiting condition    226
smoker                              155
form_filled_date                      1
dtype: int64 

Data types 
 PatientID                                   object
previous_infection                          object
test_type                                   object
date_tested                         datetime64[ns]
consultation_sought                         object
consultant_type                             object
symptoms_start_time                 datetime64[ns]
symptoms_persistance                        object
days_with_symptoms          

observations:
* there are alot of missing values 
* there a few wrong data types 
* overall data is uniform so wouldn't take much effort 
* there are questions obtained via checkboxes and need to be broken down to simpler columns

In [ ]:
Covid.columns

In [250]:
## let us rename some columns for easier wrangling:
Covid.drop("Unnamed: 0",inplace=True,axis=1)
Covid.rename(columns={"16. Did you have COVID-19 infection? if no don't proceed":"previous_infection",
                    "What type of test was used to test you for COVID-19?":"test_type",
                    "What date were you tested for COVID-19? If you don't know the exact date, please choose your best approximation.":"date_tested",
                    "Did you consult with a physician(s) for your COVID-19 symptoms? ":"consultation_sought",
                    "Who did you consult for your COVID-19 symptoms? ":"consultant_type",
                    "When did your symptoms begin? ":"symptoms_start_time",
                    '2.\tAre you still experiencing symptoms? ':"symptoms_persistance",
                    '3.\tHow many days total did you experience symptoms? ':"days_with_symptoms",
                    '4.Lifestyle & Pre-existing Conditions:            1.\tDid you have any of these pre-existing conditions/diagnoses or did you experience any of the following pre-COVID-19?':"pre-existing conditions",
                     '6.\tDid any of your pre-existing conditions change during your COVID19 symptoms?':"change in pre-exsiting condition",
                     '7.\tIn the month before becoming ill if you were sick, or in the previous month if you were not, were you a regular, occasional, or never smoker?':"smoker",},
             inplace=True)
Covid.columns

Index(['PatientID', 'previous_infection', 'test_type', 'date_tested',
       'consultation_sought', 'consultant_type', 'symptoms_start_time',
       'symptoms_persistance', 'days_with_symptoms', 'pre-existing conditions',
       'change in pre-exsiting condition', 'smoker'],
      dtype='object')

### 2.1 Datatime fields

In [273]:
Covid["date_tested"]=pd.to_datetime(Covid["date_tested"],errors = 'coerce')
Covid["symptoms_start_time"]=pd.to_datetime(Covid["symptoms_start_time"],errors = 'coerce')

In [274]:
Covid.dtypes

PatientID                                    int64
previous_infection                          object
test_type                                   object
date_tested                         datetime64[ns]
consultation_sought                         object
consultant_type                             object
symptoms_start_time                 datetime64[ns]
symptoms_persistance                        object
days_with_symptoms                          object
pre-existing conditions                     object
change in pre-exsiting condition            object
smoker                                      object
form_filled_date                    datetime64[ns]
dtype: object

In [275]:
## days_with_symptoms needs some special treatment 
Covid.days_with_symptoms.str.lower().value_counts()

7      334
10     302
14     290
       241
15     168
5      150
20     140
30     111
4       91
12      86
3       74
6       69
8       68
21      54
9       52
18      43
2       41
11      35
13      34
16      29
17      24
25      23
60      17
45      14
1        9
22       9
19       8
40       7
90       7
35       6
28       6
24       4
27       3
50       3
180      2
120      2
37       2
0        2
150      1
39       1
100      1
31       1
567      1
365      1
340      1
26       1
52       1
58       1
76       1
Name: days_with_symptoms, dtype: int64

In [276]:
Covid.days_with_symptoms=Covid.days_with_symptoms.apply(lambda x :spell(str(x).lower().strip()))

In [277]:
Covid.days_with_symptoms.value_counts()

7      334
10     302
14     290
       241
15     168
5      150
20     140
30     111
4       91
12      86
3       74
6       69
8       68
21      54
9       52
18      43
2       41
11      35
13      34
16      29
17      24
25      23
60      17
45      14
1        9
22       9
19       8
40       7
90       7
35       6
28       6
24       4
27       3
50       3
180      2
120      2
37       2
0        2
150      1
39       1
100      1
31       1
567      1
365      1
340      1
26       1
52       1
58       1
76       1
Name: days_with_symptoms, dtype: int64

In [278]:
special_inputs=[]
for e, value in enumerate(Covid.days_with_symptoms):
    if value=="" or value =="nan":
        Covid.loc[e,"days_with_symptoms"]=np.nan
    else:
        flag=re.findall('[0-9]+',str(value))
        if len(flag) ==1:
            try:
                Covid.loc[e,"days_with_symptoms"]=int(value)
            except:
                if "day" in str(value):
                    ##remove any chars from string 
                    value= re.findall('[0-9]+',str(value))
                    Covid.loc[e,"days_with_symptoms"]=int(value[0])
                elif "mont" in str(value):
                    ##remove any chars from string 
                    value= re.findall('[0-9]+',str(value))      
                    Covid.loc[e,"days_with_symptoms"]=int(value[0])*30
                elif "week" in str(value) or "weak" in str(value):
                    ##remove any chars from string 
                    value= re.findall('[0-9]+',str(value))
                    Covid.loc[e,"days_with_symptoms"]=int(value[0])*7
        else:
            special_inputs.append(e)

        

In [279]:
Covid["form_filled_date"]=data.Timestamp
for e in special_inputs:
    value=Covid.loc[e,"days_with_symptoms"]
    if "nan" in str(value) or "asympt" in str(value) or "remember" in str(value) or "sure" in str(value) or "few" in str(value):
        Covid.loc[e,"days_with_symptoms"]=np.nan
    elif "til" in str(value):
        Covid.loc[e,"days_with_symptoms"]= (Covid.loc[e,"form_filled_date"] - Covid.loc[e,"symptoms_start_time"]).days
    

In [280]:
for i in Covid.days_with_symptoms[special_inputs].value_counts().items():
    print(i)

In [281]:
## dealing with "-" ,"or", anf "to"
for e in special_inputs:
    value=Covid.loc[e,"days_with_symptoms"]
    if "-" in str(value):
        temp=value.split("-")
        print(temp)
        try:
            Covid.loc[e,'days_with_symptoms']=int(int(temp[0])*int(temp[1])/2)
        except:
            if any("day" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[0]))*int(re.sub('\D', '', temp[1]))/2
            elif any("month" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[1]))*30
    elif "to" in str(value):
        temp=value.split("to")
        print(temp)
        try:
            Covid.loc[e,'days_with_symptoms']=int(int(temp[0])*int(temp[1])/2)
        except:
            if any("day" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[0]))*int(re.sub('\D', '', temp[1]))/2
            elif any("month" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[1]))*30
    elif "or" in str(value):
        temp=value.split("or")
        print(temp)
        try:
            Covid.loc[e,'days_with_symptoms']=int(int(temp[0])*int(temp[1])/2)
        except:
            if any("day" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[0]))*int(re.sub('\D', '', temp[1]))/2
            elif any("month" in string for string in temp):
                Covid.loc[e,'days_with_symptoms']=int(re.sub('\D', '', temp[1]))*30


                
                
                
            
            
        

In [282]:
for i in Covid.days_with_symptoms[special_inputs].value_counts().items():
    print(i)

In [283]:
for i in Covid.days_with_symptoms:
    print(i)

nan
7
nan
nan
10
18
10
nan
nan
nan
22
20
18
6
11
nan
14
10
18
19
14
nan
25
12
20
15
14
nan
15
8
14
21
18
14
nan
16
25
7
20
nan
7
5
10
nan
18
20
20
16
17
30
25
20
14
12
nan
14
16
12
14
nan
15
14
10
14
10
16
14
9
12
17
10
21
7
10
15
10
14
12
nan
20
18
7
7
21
15
17
17
10
7
12
7
10
14
30
14
11
7
340
16
14
10
7
nan
45
10
12
7
14
4
15
14
12
14
nan
14
5
30
10
14
16
7
7
14
52
13
nan
7
17
5
14
nan
nan
10
10
14
15
21
21
30
13
13
7
15
10
30
28
15
9
20
30
25
10
7
10
14
16
10
10
4
10
14
10
6
14
10
21
30
4
20
16
12
10
10
12
nan
14
10
10
17
nan
60
nan
20
27
10
nan
7
10
25
12
15
nan
6
10
5
15
4
11
nan
24
20
30
30
30
30
21
30
7
567
30
8
14
4
10
14
14
14
15
30
7
12
10
7
20
2
20
30
14
20
30
7
5
14
10
10
3
14
20
20
30
18
14
10
15
15
14
20
15
30
nan
25
20
14
nan
365
30
5
15
7
10
20
14
7
14
7
7
5
20
10
11
14
14
18
22
10
7
9
nan
nan
11
3
nan
12
20
25
16
20
17
20
17
13
15
7
12
15
14
18
5
7
15
5
14
20
19
11
12
8
10
15
8
nan
15
14
7
30
7
30
2
12
nan
18
7
10
15
14
5
10
7
10
5
14
15
10
14
7
7
nan
nan
14
14
3
11
1

In [284]:
##final tweaks
for e in special_inputs:
    value=Covid.loc[e,"days_with_symptoms"]
    if "two week" in str(value):
        Covid.loc[e,"days_with_symptoms"]=14
    elif "week" in str(value):
        Covid.loc[e,"days_with_symptoms"]=7
    elif "90" in str(value):
        Covid.loc[e,"days_with_symptoms"]=90
    elif "1.5" in str(value):
        Covid.loc[e,"days_with_symptoms"]=45
    else:
        Covid.loc[e,"days_with_symptoms"]=np.nan





    

        

In [288]:
## final tweaks
for e, value in enumerate(Covid.days_with_symptoms):
    if "d" in str(value):
        Covid.loc[e,"days_with_symptoms"]=int(re.sub('\D', '', str(value)))
        #print(int(re.sub('\D', '', str(value))))


    elif "w" in str(value):
        Covid.loc[e,"days_with_symptoms"]=int(re.sub('\D', '', str(value)))*7
    elif "year" in str(value):
        Covid.loc[e,"days_with_symptoms"]=365
    


        

Covid.days_with_symptoms=Covid.days_with_symptoms.apply(lambda x :re.sub('\D', '', str(x)))


In [293]:

for e,value in enumerate(Covid.days_with_symptoms):
    if value == '':
        Covid.loc[e,"days_with_symptoms"]=np.nan
    else:
        Covid.loc[e,"days_with_symptoms"]=float(value)
Covid.days_with_symptoms=Covid.days_with_symptoms.astype(float)


### 2.2 Dealing with checkboxes 

In [13]:
Covid=pd.read_excel("covid.xlsx")

In [14]:
pre_conditions_options=["Anemia",
"Autism",
"Auto-immune/rheumatological conditions",
"Cancer (all types)",
"Chronic kidney disease",
"Diabetes Type 1",
"Diabetes Type 2",
"Ehlers-Danlos Syndrome (EDS)",
"Endometriosis",
"Fibromyalgia",
"IgA deficiency",
"Interstitial Cystitis (Bladder Pain Syndrome)",
"Hepatitis (A/B/C)",
"HIV",
"Mast Cell Activation Syndrome (MCAS)",
"Myalgic Encephalomyelitis / Chronic Fatigue Syndrome (ME/CFS)" ,
"Obesity",
"Postural Orthostatic Tachycardia Syndrome (POTS)",
"Recurrent bacterial infections",
"Recurrent viral infections",
"Restless leg syndrome",
"TMJ (temporomandibular joint dysfunction)",
"Vertigo",
"Vision: near-sighted/far-sighted",
"Vitamin D deficiency",
"None of the above",
"Food Allergies",
"Environmental Allergies (dust, mold)",
"Chemical Allergies Seasonal Allergies",
"Allergies of unknown origin",
"Other allergies",
"Insomnia",
"Lucid dreams (dreams where you are aware you are dreaming or have some control over what you dream)",
"Nightmares",
"Vivid dreams",
"Night sweats",
"Sleep apnea",
"Acid Reflux Disease",
"Celiac Disease",
"Crohn's Disease",
"Ulcerative Colitis",
"Irritable Bowel Syndrome (IBS)",
"Other GI issues",
"Asthma",
"Chronic Obstructive Pulmonary Disease (COPD)" ,
"Tuberculosis",
"Eczema",
"Viral skin conditions (cold sores, herpes, warts, molluscum) ",
"Dementia",
"Seizures/epilepsy",
"Migraine",
"Amyotrophic lateral sclerosis",
"Parkinson's disease",
"Multiple Sclerosis",
"Peripheral neuropathy",
"Coronary Heart Disease",
"Heart failure",
"Hypertension (high blood pressure)",
"Hypotension (low blood pressure)",
"History of blood clotting",
"History of strokes",
"High cholesterol/hyperlipidemia",
"Mitral valve prolapse"]

pre_conditions_options=[remove_brackets(x) for x in pre_conditions_options]
pre_conditions_options=[x.strip() for x in pre_conditions_options]
pre_conditions_options

['Anemia',
 'Autism',
 'Auto-immune/rheumatological conditions',
 'Cancer',
 'Chronic kidney disease',
 'Diabetes Type 1',
 'Diabetes Type 2',
 'Ehlers-Danlos Syndrome',
 'Endometriosis',
 'Fibromyalgia',
 'IgA deficiency',
 'Interstitial Cystitis',
 'Hepatitis',
 'HIV',
 'Mast Cell Activation Syndrome',
 'Myalgic Encephalomyelitis / Chronic Fatigue Syndrome',
 'Obesity',
 'Postural Orthostatic Tachycardia Syndrome',
 'Recurrent bacterial infections',
 'Recurrent viral infections',
 'Restless leg syndrome',
 'TMJ',
 'Vertigo',
 'Vision: near-sighted/far-sighted',
 'Vitamin D deficiency',
 'None of the above',
 'Food Allergies',
 'Environmental Allergies',
 'Chemical Allergies Seasonal Allergies',
 'Allergies of unknown origin',
 'Other allergies',
 'Insomnia',
 'Lucid dreams',
 'Nightmares',
 'Vivid dreams',
 'Night sweats',
 'Sleep apnea',
 'Acid Reflux Disease',
 'Celiac Disease',
 "Crohn's Disease",
 'Ulcerative Colitis',
 'Irritable Bowel Syndrome',
 'Other GI issues',
 'Asthma',
 

In [15]:
Covid["pre-existing conditions"]=Covid["pre-existing conditions"].apply(remove_brackets)
Covid["pre-existing conditions"]=Covid["pre-existing conditions"].str.replace(" ","")

In [16]:
for i in Covid["pre-existing conditions"].unique():
    print(i,"\n")

Nightmares 

Environmental Allergies , Chemical Allergies Seasonal Allergies 

Insomnia, Migraine 

Anemia 

Environmental Allergies , Lucid dreams , Nightmares, Vivid dreams, Vision: near-sighted/far-sighted 

Chemical Allergies Seasonal Allergies, Anemia, Vision: near-sighted/far-sighted, Vitamin D deficiency 

Insomnia 

Acid Reflux Disease, Other GI issues, High cholesterol/hyperlipidemia 

Environmental Allergies , Vitamin D deficiency 

nan 

Nightmares, Night sweats, Eczema, Migraine, Fibromyalgia 

Vision: near-sighted/far-sighted 

Food Allergies, Allergies of unknown origin, Acid Reflux Disease, Celiac Disease, Irritable Bowel Syndrome , Anemia, Vitamin D deficiency 

None of the above 

Hypertension , High cholesterol/hyperlipidemia 

Lucid dreams , Migraine, Hypertension , High cholesterol/hyperlipidemia, Anemia, Obesity, Vision: near-sighted/far-sighted, Vitamin D deficiency 

Migraine, Diabetes Type 2 

Other allergies, Night sweats, Acid Reflux Disease, Irritable Bowel S

Acid Reflux Disease, Vitamin D deficiency 

Environmental Allergies , Asthma, Anemia, Vitamin D deficiency 

Asthma, Vision: near-sighted/far-sighted 

Insomnia, Vision: near-sighted/far-sighted 

Anemia, None of the above, Anemia 

Hypertension , History of blood clotting, History of strokes 

Allergies of unknown origin, Allergic rhinitis 

Migraine, Vertigo 

Migraine, None of the above 

Chemical Allergies Seasonal Allergies, Vision: near-sighted/far-sighted 

Food Allergies, Environmental Allergies , Chemical Allergies Seasonal Allergies, Coronary Heart Disease 

Migraine, Vitamin D deficiency 

Lucid dreams , Acid Reflux Disease, Vitamin D deficiency 

Chemical Allergies Seasonal Allergies, Hypertension , Vitamin D deficiency 

Irritable Bowel Syndrome , Other GI issues, Vision: near-sighted/far-sighted, Vitamin D deficiency 

Acid Reflux Disease, Cervicalgia and BPPV 

Vision: near-sighted/far-sighted, BPH, rheumatism 

Insomnia, Obesity 

Food Allergies, Chemical Allergies Seas

In [17]:
Covid["weird_input"]=np.nan
for e,v in enumerate(Covid["pre-existing conditions"]):
    value=str(v).split(",")
    value=[v.strip() for v in value]
    for i in value:
        if i not in pre_conditions_options:
            Covid.loc[e,"weird_input"]=i
            value.remove(i)
            
    value=",".join(value)
    print(value)
    Covid.loc[e,"pre-existing conditions"]=value


Nightmares
Environmental Allergies,Chemical Allergies Seasonal Allergies
Insomnia,Migraine
Anemia
Environmental Allergies,Lucid dreams,Nightmares,Vivid dreams,Vision: near-sighted/far-sighted
Chemical Allergies Seasonal Allergies,Anemia,Vision: near-sighted/far-sighted,Vitamin D deficiency
Insomnia
Acid Reflux Disease,Other GI issues,High cholesterol/hyperlipidemia
Environmental Allergies,Vitamin D deficiency

Nightmares,Night sweats,Eczema,Migraine,Fibromyalgia
Vision: near-sighted/far-sighted
Food Allergies,Allergies of unknown origin,Acid Reflux Disease,Celiac Disease,Irritable Bowel Syndrome,Anemia,Vitamin D deficiency
None of the above



Hypertension,High cholesterol/hyperlipidemia
Lucid dreams,Migraine,Hypertension,High cholesterol/hyperlipidemia,Anemia,Obesity,Vision: near-sighted/far-sighted,Vitamin D deficiency
Migraine,Diabetes Type 2
Other allergies,Night sweats,Acid Reflux Disease,Irritable Bowel Syndrome,Vitamin D deficiency
None of the above
Environmental Allergies,Insom

Diabetes Type 2
Insomnia,Acid Reflux Disease,Hypertension,Vision: near-sighted/far-sighted
None of the above
Asthma
Insomnia,Irritable Bowel Syndrome,Migraine,Anemia
Insomnia,Nightmares,Chronic kidney disease
Obesity

Environmental Allergies,Irritable Bowel Syndrome,Vitamin D deficiency
None of the above
Irritable Bowel Syndrome,Migraine,Hypotension,Vision: near-sighted/far-sighted
Environmental Allergies,Chemical Allergies Seasonal Allergies,Celiac Disease,Vision: near-sighted/far-sighted,Vitamin D deficiency
Hypertension,History of blood clotting
Heart failure,Hypertension,Diabetes Type 2
Anemia
Nightmares,Hypotension,B12 deficiency
Obesity

Obesity

Insomnia

None of the above
None of the above
Anemia
High cholesterol/hyperlipidemia

Hypertension,Chronic kidney disease
None of the above

Other allergies

None of the above
Obesity

Diabetes Type 2
None of the above
Nightmares,Vision: near-sighted/far-sighted
Insomnia,Irritable Bowel Syndrome,High cholesterol/hyperlipidemia,Auto-immun


Environmental Allergies,Eczema
Vision: near-sighted/far-sighted,None of the above
Migraine
Sleep apnea,Acid Reflux Disease
Hypertension
Irritable Bowel Syndrome,Other GI issues,High cholesterol/hyperlipidemia

Food Allergies,Acid Reflux Disease
Insomnia,Night sweats,Asthma,Migraine,Anemia

Lucid dreams,Nightmares,Diabetes Type 1

Nightmares,Vivid dreams,Viral skin conditions
None of the above

Environmental Allergies,Sleep apnea,Other GI issues,Asthma,Migraine
Recurrent bacterial infections,Recurrent viral infections
Acid Reflux Disease,Obesity
Irritable Bowel Syndrome

Diabetes Type 1
None of the above
None of the above


None of the above
Migraine
Migraine
Environmental Allergies,Irritable Bowel Syndrome
None of the above
Environmental Allergies,Nightmares

Night sweats,Irritable Bowel Syndrome,Recurrent bacterial infections,Recurrent viral infections
Allergies of unknown origin,Obesity

Insomnia,Migraine
Acid Reflux Disease,Irritable Bowel Syndrome
None of the above
Hypertension,Di

None of the above
None of the above
None of the above

None of the above
Food Allergies,Environmental Allergies
None of the above

None of the above
Irritable Bowel Syndrome
Allergies of unknown origin,Vision: near-sighted/far-sighted
Sleep apnea,Asthma,Vitamin D deficiency,Rheumatoid Arthritis
None of the above
None of the above
Hypertension,High cholesterol/hyperlipidemia,Diabetes Type 2
Vision: near-sighted/far-sighted
None of the above
Food Allergies


Eczema
Chemical Allergies Seasonal Allergies,Other allergies
None of the above
Other GI issues,Anemia,Auto-immune/rheumatological conditions,Postural Orthostatic Tachycardia Syndrome
Hypertension
None of the above
Diabetes Type 2
Lucid dreams,Acid Reflux Disease,Vertigo,Vision: near-sighted/far-sighted
None of the above
Recurrent bacterial infections
Other GI issues
Diabetes Type 2
Environmental Allergies,Allergies of unknown origin,Hypotension,Recurrent viral infections
Recurrent bacterial infections
Food Allergies,Allergies of unkn

In [23]:
temp=Covid["pre-existing conditions"].str.get_dummies(sep=",")
#Covid.drop("pre-existing conditions",inplace=True)
Covid=pd.concat([Covid,temp],axis=1)
temp

,Acid Reflux Disease,Allergies of unknown origin,Anemia,Asthma,Autism,Auto-immune/rheumatological conditions,B12 deficiency,Cancer,Celiac Disease,Chemical Allergies Seasonal Allergies,...,inflammatory back pain,nephritis,nephrolithiasis,osteoarthritis,otitis media,polycystic ovary,rheumatism,shortness of breath,sinusitis,smell sense gone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2568,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
for i in Covid.columns:
    print(i,"\n")

PatientID 

previous_infection 

test_type 

date_tested 

consultation_sought 

consultant_type 

symptoms_start_time 

symptoms_persistance 

days_with_symptoms 

pre-existing conditions 

change in pre-exsiting condition 

smoker 

form_filled_date 

weird_input 

Acid Reflux Disease 

Allergies of unknown origin 

Anemia 

Asthma 

Autism 

Auto-immune/rheumatological conditions 

B12 deficiency 

Cancer 

Celiac Disease 

Chemical Allergies Seasonal Allergies 

Chronic Obstructive Pulmonary Disease 

Chronic kidney disease 

Coronary Heart Disease 

Crohn's Disease 

Dementia 

Diabetes Type 1 

Diabetes Type 2 

Eczema 

Ehlers-Danlos Syndrome 

Endometriosis 

Environmental Allergies 

Fibromyalgia 

Food Allergies 

H. pylori 

Heart failure 

Hepatitis 

High cholesterol/hyperlipidemia 

History of blood clotting 

History of strokes 

Hypertension 

Hypotension 

IgA deficiency 

Insomnia 

Interstitial Cystitis 

Irritable Bowel Syndrome 

Lucid dreams 

Migraine 

Mitral va

In [25]:
checkboxes=list(temp.columns)
columns=list(Covid.columns)
for col in columns:
    columns2=columns
    columns2.remove(col)
    for j in columns2:
        col2=col.lower().strip()
        j2=j.lower().strip()
        if col2==j2:
            print(f"{col}=={j}")
            Covid[col]=Covid[j].astype(float)+Covid[col].astype(float)
            Covid.drop(j,inplace=True,axis=1)
            checkboxes.remove(j)
    

Sinusitis==sinusitis


In [26]:
for i in checkboxes:
    Covid[i].fillna(0,inplace=True)

In [27]:
Covid.weird_input.value_counts()[2:12]

Hypothyroidism     5
sinusitis          4
بهاق               3
Hyperthyroidism    3
                   2
Thyroidectomy      2
Astigmatism        2
Depression         2
sour throat        1
Astrocytoma        1
Name: weird_input, dtype: int64

In [28]:
Covid.drop("weird_input",inplace=True,axis=1)

In [29]:
pre_conditions_options

['Anemia',
 'Autism',
 'Auto-immune/rheumatological conditions',
 'Cancer',
 'Chronic kidney disease',
 'Diabetes Type 1',
 'Diabetes Type 2',
 'Ehlers-Danlos Syndrome',
 'Endometriosis',
 'Fibromyalgia',
 'IgA deficiency',
 'Interstitial Cystitis',
 'Hepatitis',
 'HIV',
 'Mast Cell Activation Syndrome',
 'Myalgic Encephalomyelitis / Chronic Fatigue Syndrome',
 'Obesity',
 'Postural Orthostatic Tachycardia Syndrome',
 'Recurrent bacterial infections',
 'Recurrent viral infections',
 'Restless leg syndrome',
 'TMJ',
 'Vertigo',
 'Vision: near-sighted/far-sighted',
 'Vitamin D deficiency',
 'None of the above',
 'Food Allergies',
 'Environmental Allergies',
 'Chemical Allergies Seasonal Allergies',
 'Allergies of unknown origin',
 'Other allergies',
 'Insomnia',
 'Lucid dreams',
 'Nightmares',
 'Vivid dreams',
 'Night sweats',
 'Sleep apnea',
 'Acid Reflux Disease',
 'Celiac Disease',
 "Crohn's Disease",
 'Ulcerative Colitis',
 'Irritable Bowel Syndrome',
 'Other GI issues',
 'Asthma',
 

In [30]:
Covid[checkboxes].T

,0,1,2,3,4,5,6,7,8,9,...,2561,2562,2563,2564,2565,2566,2567,2568,2569,2570
Acid Reflux Disease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Allergies of unknown origin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Anemia,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Asthma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Autism,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
otitis media,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
polycystic ovary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rheumatism,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
shortness of breath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
Covid.shape

(2571, 93)

In [33]:
#Covid.to_excel("covid.xlsx",index=False)
Covid.to_excel("covid_dummies_new.xlsx",index=False)

## 3. Treatment DataFrame 

In [383]:
treat=pd.read_excel("treatment_related.xlsx",engine="openpyxl")

In [384]:
treat

,Unnamed: 0,PatientID,1.\tWere you hospitalized?,2.\tIf yes: how long were you hospitalized? [Number of days],3.\tDid you receive oxygen support in the hospital?,"Have you tried any of the following treatments for your COVID19 symptoms, if yes, how helpful it was? This includes Prescription or off-the-counter Medications, or Alternative Treatments.",Have you got the Covid-19 vaccine?,Type of the vaccine,How many shots have you got?,When you get the vaccine,Have you got infected with COVID-19 after vaccination?
0,0,0, No,NaN,NaN," Anti-histamines, Aspirin,  Antibiotics,  (...",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,1,2021-11-15 00:00:00,No
1,1,1, No,NaN, No," Anti-histamines,  Antibiotics",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,2,5/10/0021,Yes
2,2,2, No,NaN, No, None,Yes, CoronaVac (Sinovac),2,2021-04-03 00:00:00,Yes
3,3,3, No,Nil, No, Azithromycin,Yes, CoronaVac (Sinovac),2,2021-04-19 00:00:00,Yes
4,4,4, No,NaN, No," Azithromycin,  (Tylenol, Paracetamol),  Cu...",Yes, CoronaVac (Sinovac),2,2021-02-13 00:00:00,Yes
...,...,...,...,...,...,...,...,...,...,...,...
2566,2566,2566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2567,2567,2567, Yes,10," Yes, nasal cannula","Enoxaparin (Lovenox),  NSAIDs (Ibuprofen,Napr...",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",1,2021-09-01 00:00:00,Yes
2568,2568,2568, Yes,25," Yes, I was intubated"," Anti-histamines,  Steroids(Prednisone and D...",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",2,2021-07-11 00:00:00,No
2569,2569,2569, No,NaN, I was not hospitalized," H1 type Antihistamines, Aspirin, Antibiotics",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",2,2020-12-20 00:00:00,Yes


In [385]:
treat.columns

Index(['Unnamed: 0', 'PatientID', '1.\tWere you hospitalized?  ',
       '2.\tIf yes: how long were you hospitalized? [Number of days] ',
       '3.\tDid you receive oxygen support in the hospital? ',
       'Have you tried any of the following treatments for your COVID19 symptoms, if yes, how helpful it was? This includes Prescription or off-the-counter Medications, or Alternative Treatments.',
       'Have you got the Covid-19 vaccine?', 'Type of the vaccine',
       'How many shots have you got?', 'When you get the vaccine',
       'Have you got infected with COVID-19 after vaccination?'],
      dtype='object')

In [386]:
## let us rename some columns for easier wrangling:
#treat.drop("Unnamed: 0",inplace=True,axis=1)
treat.PatientID=treat.PatientID.astype(str)
treat.rename(columns={'1.\tWere you hospitalized?  ': "hospitalized",
                    "What type of test was used to test you for COVID-19?":"test_type",
                    "2.\tIf yes: how long were you hospitalized? [Number of days] ":"hospitlization_duration",
                    "3.\tDid you receive oxygen support in the hospital? ":"oxygen_support",
                    "Who did you consult for your COVID-19 symptoms? ":"consultant_type",
                    "Have you tried any of the following treatments for your COVID19 symptoms, if yes, how helpful it was? This includes Prescription or off-the-counter Medications, or Alternative Treatments.":"OTC_prescription_alternate_use",
                    'Have you got the Covid-19 vaccine?':"vaccine",
                      "How many shots have you got?":"vaccine_shots",
                      'When you get the vaccine':"vaccine_date",
                    'Have you got infected with COVID-19 after vaccination?':"infection_post_vaccination"},
             inplace=True)
treat.columns

Index(['Unnamed: 0', 'PatientID', 'hospitalized', 'hospitlization_duration',
       'oxygen_support', 'OTC_prescription_alternate_use', 'vaccine',
       'Type of the vaccine', 'vaccine_shots', 'vaccine_date',
       'infection_post_vaccination'],
      dtype='object')

In [387]:
treat.head()

,Unnamed: 0,PatientID,hospitalized,hospitlization_duration,oxygen_support,OTC_prescription_alternate_use,vaccine,Type of the vaccine,vaccine_shots,vaccine_date,infection_post_vaccination
0,0,0, No,NaN,NaN," Anti-histamines, Aspirin,  Antibiotics,  (...",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,1,2021-11-15 00:00:00,No
1,1,1, No,NaN, No," Anti-histamines,  Antibiotics",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,2,5/10/0021,Yes
2,2,2, No,NaN, No, None,Yes, CoronaVac (Sinovac),2,2021-04-03 00:00:00,Yes
3,3,3, No,Nil, No, Azithromycin,Yes, CoronaVac (Sinovac),2,2021-04-19 00:00:00,Yes
4,4,4, No,NaN, No," Azithromycin,  (Tylenol, Paracetamol),  Cu...",Yes, CoronaVac (Sinovac),2,2021-02-13 00:00:00,Yes


In [388]:
full_report(treat)

Missing Values
 Unnamed: 0                           0
PatientID                            0
hospitalized                       138
hospitlization_duration           2069
oxygen_support                     531
OTC_prescription_alternate_use     177
vaccine                            133
Type of the vaccine                729
vaccine_shots                      695
vaccine_date                       804
infection_post_vaccination         666
dtype: int64 

Data types 
 Unnamed: 0                         int64
PatientID                         object
hospitalized                      object
hospitlization_duration           object
oxygen_support                    object
OTC_prescription_alternate_use    object
vaccine                           object
Type of the vaccine               object
vaccine_shots                     object
vaccine_date                      object
infection_post_vaccination        object
dtype: object 

 Quick Description 
 <bound method NDFrame.describe of      

same usual observations 
* some wrong datatypes 
* vaccine_shots and hospitlization duration need cleaning 
* OTC_presciption_alternate should be transformed to dummy columns

In [389]:
treat.vaccine_date=pd.to_datetime(treat.vaccine_date,errors = 'coerce')

In [390]:
treat.vaccine_shots=treat.vaccine_shots.str.lower()
treat.vaccine_shots.value_counts()
    

2 shots                                    36
two                                        19
one                                         9
1 shot                                      6
2 shot                                      5
02                                          4
3 shots                                     4
twice                                       3
2shot                                       3
two                                         3
none                                        2
three                                       2
twice                                       2
2 plus booster                              2
once                                        2
one shot                                    2
gilgit                                      1
2 the last one was in the date bellow       1
2 sinopharm 2 pfizer                        1
2 the last one in the history bellow        1
2 for vaccine and one booster of pfizer     1
single                            

In [394]:

for e, value in enumerate(treat.vaccine_shots):
    if "booster" in str(value):
        booster=1
    else:
        booster=0
    flag=re.findall('[0-9]+',str(value))
    if len(flag)==1:
        treat.loc[e,'vaccine_shots']=int(flag[0])+booster
    elif len(flag)==2:
        treat.loc[e,'vaccine_shots']=int(flag[0])+int(flag[1])+booster
    elif len(flag)==0:
        ## no booster
        if "two" in str(value) or "twice" in str(value) or "جرعتان" in str(value):
            treat.loc[e,'vaccine_shots']=2+booster
        elif "one" in str(value) or "single" in str(value) or "once" in str(value):
            treat.loc[e,'vaccine_shots']=1+booster
        elif "three" in str(value):
            treat.loc[e,'vaccine_shots']=3+booster
        else:
            treat.loc[e,'vaccine_shots']=None
            
            




            
        
        
 

In [395]:
treat.vaccine_shots.value_counts()

2    88
1    25
3    15
4     5
Name: vaccine_shots, dtype: int64

In [398]:
### now with hospitlization duration 
treat.hospitlization_duration=treat.hospitlization_duration.str.lower()
treat.hospitlization_duration.value_counts()

no               73
none             15
10 days          11
7 days            9
14 days           7
                 ..
zero days         1
14 day            1
14days            1
16 d              1
less than day     1
Name: hospitlization_duration, Length: 65, dtype: int64

In [413]:
special_inputs=[]
for e, value in enumerate(treat.hospitlization_duration):
    if "nan" in str(value) or  "nil" in str(value) or "no" in str(value) or "na" in str(value):
        treat.loc[e,"hospitlization_duration"]=np.nan
        
    else:
        flag=re.findall('[0-9]+',str(value))
        if len(flag)==1:
            if "day" in str(value):
                treat.loc[e,"hospitlization_duration"]=int(flag[0])
            elif "mont" in str(value):
                treat.loc[e,"hospitlization_duration"]=int(flag[0])*30
            elif "week" in str(value):
                treat.loc[e,"hospitlization_duration"]=int(flag[0])*7
            else:
                try:
                    int(value)
                except:
                    special_inputs.append(e)
                
        else :
            special_inputs.append(e)

                
        

In [415]:
## final tweaks
for e in special_inputs:
    value=str(treat.loc[e,"hospitlization_duration"])
    if "." in value or  "-" in value or "_" in value:
        treat.loc[e,"hospitlization_duration"]=np.nan
    elif ("less than" in value and "day" in value) or "zero" in value or "wasn't" in value or "couples of hours":
        treat.loc[e,"hospitlization_duration"]=0
    elif ("two" in value and "day" in value):
        treat.loc[e,"hospitlization_duration"]=2
    elif ("one" in value and "day" in value):
        treat.loc[e,"hospitlization_duration"]=1
    elif "4" in value:
        treat.loc[e,"hospitlization_duration"]=4
    elif "16 d" in value:
        treat.loc[e,"hospitlization_duration"]=16



In [416]:
treat.hospitlization_duration.astype(float)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2566   NaN
2567   NaN
2568   NaN
2569   NaN
2570   NaN
Name: hospitlization_duration, Length: 2571, dtype: float64

In [49]:
## save before creating dummy columns 
#treat.to_excel("treatment_related.xlsx",index=False)

treat=pd.read_excel("treatment_related.xlsx")
treat

,Unnamed: 0,PatientID,hospitalized,hospitlization_duration,oxygen_support,OTC_prescription_alternate_use,vaccine,Type of the vaccine,vaccine_shots,vaccine_date,infection_post_vaccination
0,0,0, No,NaN,NaN," Anti-histamines, Aspirin,  Antibiotics,  (...",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,NaN,2021-11-15,No
1,1,1, No,NaN, No," Anti-histamines,  Antibiotics",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,NaN,NaT,Yes
2,2,2, No,NaN, No, None,Yes, CoronaVac (Sinovac),NaN,2021-04-03,Yes
3,3,3, No,NaN, No, Azithromycin,Yes, CoronaVac (Sinovac),NaN,2021-04-19,Yes
4,4,4, No,NaN, No," Azithromycin,  (Tylenol, Paracetamol),  Cu...",Yes, CoronaVac (Sinovac),NaN,2021-02-13,Yes
...,...,...,...,...,...,...,...,...,...,...,...
2566,2566,2566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2567,2567,2567, Yes,NaN," Yes, nasal cannula","Enoxaparin (Lovenox),  NSAIDs (Ibuprofen,Napr...",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2021-09-01,Yes
2568,2568,2568, Yes,NaN," Yes, I was intubated"," Anti-histamines,  Steroids(Prednisone and D...",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2021-07-11,No
2569,2569,2569, No,NaN, I was not hospitalized," H1 type Antihistamines, Aspirin, Antibiotics",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2020-12-20,Yes


In [50]:
"(Diphenhydramine, acrivastine, and cetirizine, like Benadryl, Zyrtec, Claritin)" "--> H1 type Antihistamines"
"(Tylenol, Paracetamol)" "--->. NSAIDs "

'(Tylenol, Paracetamol)--->. NSAIDs '

In [51]:
known_meds=["Direct oral anticoagulants",
            
"Rivaroxaban (Xarelto)",
            
"Warfarin (Coumadin)",
"Anti-inflammatories",
            
"Curcumin (tumeric)",
            
"Omega 3 / DHA / EPA (Fish oil) ",
            
"Intravenous gamma globulin",
            
"Convalescent plasma",
            
"None",
            
"Others (specify)",
            
"Anti-histamines",
            
"H1 type Antihistamines",
              
"H2 type Antihistamines (cimetidine,famotidine, like Pepcid)",
"Products Containing Cannabis or Cannabis-derived Compounds, Including delta-9- tetrahydrocannabinol (THC) and cannabidiol (CBD)",
            
            
"Steroids(Prednisone and Dexamethasone etc) ",
"Apixaban (Eliquis)",
"Dabigatran (Pradaxa)",
"Dalteparin (Fragmin)",
"Edoxaban (Savaysa)",
"Enoxaparin (Lovenox)",
"Fondaparinux (Arixtra)",
"Heparin (Innohep)",
"Rivaroxaban (Xarelto)",
"Warfarin (Coumadin, Jantoven) ",
"Aspirin",
"Cilostazol",
"Clopidogrel (Plavix) ",
"Dipyridamole (Persantine) ",
"Eptifibatide (Integrilin) ",
"Prasugrel (Effient) ",
"Ticagrelor (Brilinta) ",
"Tirofiban (Aggrastat) ",
"Vorapaxar (Zontivity)",
"Remsdesevir",
"Veklury (an antiviral medicine used to treat coronavirus disease)",
"Antibiotics",
"Azithromycin",
"Malaria treatments",
"Chloroquine",
"Hydroxychloroquine",
"Anti-oxidants",
"Oxaloacetate",
"Over the counter painkillers",
"Non-NSAIDs",

"NSAIDs (Ibuprofen,Naproxen, Adult aspirin )"]


known_meds=[remove_brackets(x) for x in known_meds]
known_meds=[x.strip() for x in known_meds]
known_meds

['Direct oral anticoagulants',
 'Rivaroxaban',
 'Warfarin',
 'Anti-inflammatories',
 'Curcumin',
 'Omega 3 / DHA / EPA',
 'Intravenous gamma globulin',
 'Convalescent plasma',
 'None',
 'Others',
 'Anti-histamines',
 'H1 type Antihistamines',
 'H2 type Antihistamines',
 'Products Containing Cannabis or Cannabis-derived Compounds, Including delta-9- tetrahydrocannabinol  and cannabidiol',
 'Steroids',
 'Apixaban',
 'Dabigatran',
 'Dalteparin',
 'Edoxaban',
 'Enoxaparin',
 'Fondaparinux',
 'Heparin',
 'Rivaroxaban',
 'Warfarin',
 'Aspirin',
 'Cilostazol',
 'Clopidogrel',
 'Dipyridamole',
 'Eptifibatide',
 'Prasugrel',
 'Ticagrelor',
 'Tirofiban',
 'Vorapaxar',
 'Remsdesevir',
 'Veklury',
 'Antibiotics',
 'Azithromycin',
 'Malaria treatments',
 'Chloroquine',
 'Hydroxychloroquine',
 'Anti-oxidants',
 'Oxaloacetate',
 'Over the counter painkillers',
 'Non-NSAIDs',
 'NSAIDs']

In [52]:
treat.columns

Index(['Unnamed: 0', 'PatientID', 'hospitalized', 'hospitlization_duration',
       'oxygen_support', 'OTC_prescription_alternate_use', 'vaccine',
       'Type of the vaccine', 'vaccine_shots', 'vaccine_date',
       'infection_post_vaccination'],
      dtype='object')

In [53]:
treat["OTC_prescription_alternate_use"].value_counts()

 None                                                                               182
 (Tylenol, Paracetamol)                                                             109
 Azithromycin,  (Tylenol, Paracetamol)                                              52
 Antibiotics                                                                         44
 Antibiotics,  Azithromycin                                                         43
                                                                                    ... 
 Azithromycin,  (Tylenol, Paracetamol), It was helpful for temprature reduction      1
Aspirin,  Antibiotics, zinc                                                           1
 Antibiotics, zinc                                                                    1
 H1 type Antihistamines, Dabigatran (Pradaxa)                                         1
Enoxaparin (Lovenox), Favipiravine 200                                                 1
Name: OTC_prescriptio

In [54]:
treat["OTC_prescription_alternate_use"]=treat["OTC_prescription_alternate_use"].str.replace("\(full dose\)","")
treat["OTC_prescription_alternate_use"]=treat["OTC_prescription_alternate_use"].str.replace("\(Diphenhydramine, acrivastine, and cetirizine, like Benadryl, Zyrtec, Claritin\)","H1 type Antihistamines")
treat["OTC_prescription_alternate_use"]=treat["OTC_prescription_alternate_use"].str.replace("\(Tylenol, Paracetamol\)","NSAIDs")






In [55]:
treat["OTC_prescription_alternate_use"]=treat["OTC_prescription_alternate_use"].str.replace("used to treat coronavirus disease\)","used to treat coronavirus disease")



In [56]:
treat["OTC_prescription_alternate_use"]=treat["OTC_prescription_alternate_use"].apply(remove_brackets)

In [57]:
treat["weird_input"]=np.nan
for e,v in enumerate(treat["OTC_prescription_alternate_use"]):
    value=str(v).split(",")
    print(value)
    value=[v.replace(" ","") for v in value]
    print(value)
    value=[v.strip() for v in value]
    print(value)
    value=list(set(value))
    value_2=[]
    for i in value:
        if i in known_meds:
            value_2.append(i)
        
        else:
            treat.loc[e,"weird_input"]=i
    
    value=",".join(value_2)
    print("\n")
    print(value)
    print("\n")
    treat.loc[e,"OTC_prescription_alternate_use"]=value

['\uf071 Anti-histamines', ' Aspirin', ' \uf071 Antibiotics', ' \uf071 NSAIDs', ' Ascorbic acid - intestinal antiseptic']
['Anti-histamines', ' Aspirin', ' Antibiotics', ' NSAIDs', ' Ascorbic acid - intestinal antiseptic']
['Anti-histamines', 'Aspirin', 'Antibiotics', 'NSAIDs', 'Ascorbic acid - intestinal antiseptic']


Antibiotics,Aspirin,Anti-histamines,NSAIDs


['\uf071 Anti-histamines', ' \uf071 Antibiotics']
['Anti-histamines', ' Antibiotics']
['Anti-histamines', 'Antibiotics']


Anti-histamines,Antibiotics


['\uf071 None']
['None']
['None']


None


['\uf071 Azithromycin']
['Azithromycin']
['Azithromycin']


Azithromycin


['\uf071 Azithromycin', ' \uf071 NSAIDs', ' \uf071 Curcumin ', ' Others ', ' Ivermite', ' vit c', ' Calcium supplements ']
['Azithromycin', ' NSAIDs', ' Curcumin ', ' Others ', ' Ivermite', ' vit c', ' Calcium supplements ']
['Azithromycin', 'NSAIDs', 'Curcumin', 'Others', 'Ivermite', 'vit c', 'Calcium supplements']


Curcumin,Others,Azithromycin,NSAIDs


['\u

['\uf071 Steroids', ' \uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 NSAIDs']
['Steroids', ' Antibiotics', ' Azithromycin', ' NSAIDs']
['Steroids', 'Antibiotics', 'Azithromycin', 'NSAIDs']


Steroids,Antibiotics,NSAIDs,Azithromycin


['\uf071 Antibiotics', ' \uf071 NSAIDs']
['Antibiotics', ' NSAIDs']
['Antibiotics', 'NSAIDs']


Antibiotics,NSAIDs


['\uf071 Anti-histamines', ' \uf071 Antibiotics', ' \uf071 NSAIDs', ' Vit-C']
['Anti-histamines', ' Antibiotics', ' NSAIDs', ' Vit-C']
['Anti-histamines', 'Antibiotics', 'NSAIDs', 'Vit-C']


Anti-histamines,Antibiotics,NSAIDs


['\uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 NSAIDs']
['Antibiotics', ' Azithromycin', ' NSAIDs']
['Antibiotics', 'Azithromycin', 'NSAIDs']


Antibiotics,NSAIDs,Azithromycin


['Heparin ', ' \uf071 Antibiotics']
['Heparin ', ' Antibiotics']
['Heparin', 'Antibiotics']


Heparin,Antibiotics


['Aspirin', ' Clopidogrel ', ' \uf071 Antibiotics', ' \uf071 Azithromycin']
['Aspirin', ' Clopidogrel ', ' Anti


None


['Aspirin', ' \uf071 Antibiotics', ' Others ', ' Vit c']
['Aspirin', ' Antibiotics', ' Others ', ' Vit c']
['Aspirin', 'Antibiotics', 'Others', 'Vit c']


Aspirin,Antibiotics,Others


['\uf071 NSAIDs', ' \uf071 Omega 3 / DHA / EPA ', ' Zinc', ' vitamin c ', ' multivitamins']
['NSAIDs', ' Omega 3 / DHA / EPA ', ' Zinc', ' vitamin c ', ' multivitamins']
['NSAIDs', 'Omega 3 / DHA / EPA', 'Zinc', 'vitamin c', 'multivitamins']


Omega 3 / DHA / EPA,NSAIDs


['\uf071 NSAIDs', ' \uf071 NSAIDs ']
['NSAIDs', ' NSAIDs ']
['NSAIDs', 'NSAIDs']


NSAIDs


['nan']
['nan']
['nan']





['nan']
['nan']
['nan']





['\uf071 None']
['None']
['None']


None


['\uf071 NSAIDs', ' Multivitamins']
['NSAIDs', ' Multivitamins']
['NSAIDs', 'Multivitamins']


NSAIDs


['\uf071 NSAIDs', ' Vitamin C and D ', 'zinc']
['NSAIDs', ' Vitamin C and D ', 'zinc']
['NSAIDs', 'Vitamin C and D', 'zinc']


NSAIDs


['\uf071 NSAIDs']
['NSAIDs']
['NSAIDs']


NSAIDs


['\uf071 NSAIDs']
['NSAIDs']
['NSAIDs']


NSAIDs




['Surbex z and Cac1000']
['Surbex z and Cac1000']





['\uf071 Anti-histamines', ' H1 type Antihistamines', ' \uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 Non-NSAIDs', ' \uf071 NSAIDs']
['Anti-histamines', ' H1 type Antihistamines', ' Antibiotics', ' Azithromycin', ' Non-NSAIDs', ' NSAIDs']
['Anti-histamines', 'H1 type Antihistamines', 'Antibiotics', 'Azithromycin', 'Non-NSAIDs', 'NSAIDs']


Non-NSAIDs,Azithromycin,Antibiotics,H1 type Antihistamines,Anti-histamines,NSAIDs


['\uf071 NSAIDs']
['NSAIDs']
['NSAIDs']


NSAIDs


['\uf071 Antibiotics', ' \uf071 Azithromycin']
['Antibiotics', ' Azithromycin']
['Antibiotics', 'Azithromycin']


Antibiotics,Azithromycin


['\uf071 Steroids', ' \uf071 Antibiotics', ' \uf071 Hydroxychloroquine', ' \uf071 Direct oral anticoagulants']
['Steroids', ' Antibiotics', ' Hydroxychloroquine', ' Direct oral anticoagulants']
['Steroids', 'Antibiotics', 'Hydroxychloroquine', 'Direct oral anticoagulants']


Steroids,Antibiotics,Direct oral anticoagul

Anti-histamines,Antibiotics,NSAIDs,Azithromycin


['\uf071 None']
['None']
['None']


None


['\uf071 NSAIDs ']
['NSAIDs ']
['NSAIDs']


NSAIDs


['\uf071 H2 type Antihistamines ', ' Heparin ', ' Warfarin ', ' \uf071 Remsdesevir', ' \uf071 Antibiotics', ' \uf071 Hydroxychloroquine', ' \uf071 Anti-oxidants', ' \uf071 NSAIDs ', ' \uf071 Warfarin ']
['H2 type Antihistamines ', ' Heparin ', ' Warfarin ', ' Remsdesevir', ' Antibiotics', ' Hydroxychloroquine', ' Anti-oxidants', ' NSAIDs ', ' Warfarin ']
['H2 type Antihistamines', 'Heparin', 'Warfarin', 'Remsdesevir', 'Antibiotics', 'Hydroxychloroquine', 'Anti-oxidants', 'NSAIDs', 'Warfarin']


H2 type Antihistamines,Warfarin,Antibiotics,Remsdesevir,Heparin,Anti-oxidants,NSAIDs,Hydroxychloroquine


['\uf071 Anti-histamines', ' H1 type Antihistamines', ' \uf071 H2 type Antihistamines ', ' \uf071 Steroids']
['Anti-histamines', ' H1 type Antihistamines', ' H2 type Antihistamines ', ' Steroids']
['Anti-histamines', 'H1 type Antihistamines', 'H2 t

...


Anti-histamines,NSAIDs


['\uf071 Over the counter painkillers', ' \uf071 Non-NSAIDs', ' \uf071 NSAIDs', ' \uf071 NSAIDs ']
['Over the counter painkillers', ' Non-NSAIDs', ' NSAIDs', ' NSAIDs ']
['Over the counter painkillers', 'Non-NSAIDs', 'NSAIDs', 'NSAIDs']


NSAIDs,Non-NSAIDs,Over the counter painkillers


['\uf071 Anti-histamines', ' \uf071 H1 type Antihistamines', ' \uf071 Steroids', ' \uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 NSAIDs ']
['Anti-histamines', ' H1 type Antihistamines', ' Steroids', ' Antibiotics', ' Azithromycin', ' NSAIDs ']
['Anti-histamines', 'H1 type Antihistamines', 'Steroids', 'Antibiotics', 'Azithromycin', 'NSAIDs']


Azithromycin,Antibiotics,H1 type Antihistamines,Steroids,Anti-histamines,NSAIDs


['\uf071 Anti-histamines', ' \uf071 NSAIDs']
['Anti-histamines', ' NSAIDs']
['Anti-histamines', 'NSAIDs']


Anti-histamines,NSAIDs


['\uf071 NSAIDs']
['NSAIDs']
['NSAIDs']


NSAIDs


['\uf071 NSAIDs']
['NSAIDs']
['NSAIDs']


NSAIDs


['\uf071 Anti-hi

Heparin,Aspirin,Azithromycin


['\uf071 Antibiotics', ' \uf071 Azithromycin']
['Antibiotics', ' Azithromycin']
['Antibiotics', 'Azithromycin']


Antibiotics,Azithromycin


['\uf071 None']
['None']
['None']


None


['\uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 NSAIDs']
['Antibiotics', ' Azithromycin', ' NSAIDs']
['Antibiotics', 'Azithromycin', 'NSAIDs']


Antibiotics,NSAIDs,Azithromycin


['H1 type Antihistamines', ' \uf071 NSAIDs']
['H1 type Antihistamines', ' NSAIDs']
['H1 type Antihistamines', 'NSAIDs']


H1 type Antihistamines,NSAIDs


['H1 type Antihistamines', ' \uf071 NSAIDs']
['H1 type Antihistamines', ' NSAIDs']
['H1 type Antihistamines', 'NSAIDs']


H1 type Antihistamines,NSAIDs


['H1 type Antihistamines', ' \uf071 NSAIDs']
['H1 type Antihistamines', ' NSAIDs']
['H1 type Antihistamines', 'NSAIDs']


H1 type Antihistamines,NSAIDs


['\uf071 Remsdesevir', ' \uf071 Direct oral anticoagulants']
['Remsdesevir', ' Direct oral anticoagulants']
['Remsdesevir', 'Direct oral

NSAIDs


['\uf071 Anti-histamines', ' antiviral', ' antitussive', ' ']
['Anti-histamines', ' antiviral', ' antitussive', ' ']
['Anti-histamines', 'antiviral', 'antitussive', '']


Anti-histamines


['\uf071 Anti-histamines', ' \uf071 H1 type Antihistamines', ' H1 type Antihistamines', ' \uf071 Antibiotics', ' \uf071 NSAIDs']
['Anti-histamines', ' H1 type Antihistamines', ' H1 type Antihistamines', ' Antibiotics', ' NSAIDs']
['Anti-histamines', 'H1 type Antihistamines', 'H1 type Antihistamines', 'Antibiotics', 'NSAIDs']


Anti-histamines,H1 type Antihistamines,NSAIDs,Antibiotics


['H1 type Antihistamines', ' \uf071 Antibiotics', ' \uf071 NSAIDs']
['H1 type Antihistamines', ' Antibiotics', ' NSAIDs']
['H1 type Antihistamines', 'Antibiotics', 'NSAIDs']


Antibiotics,H1 type Antihistamines,NSAIDs


['\uf071 Anti-histamines', ' H1 type Antihistamines', ' \uf071 Oxaloacetate', ' \uf071 Over the counter painkillers', ' \uf071 NSAIDs']
['Anti-histamines', ' H1 type Antihistamines', ' Oxaloace

['H1 type Antihistamines', ' H2 type Antihistamines ', ' Curcumin ']
['H1 type Antihistamines', 'H2 type Antihistamines', 'Curcumin']


H1 type Antihistamines,Curcumin,H2 type Antihistamines


['\uf071 Anti-histamines']
['Anti-histamines']
['Anti-histamines']


Anti-histamines


['\uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 Hydroxychloroquine', ' \uf071 Anti-oxidants']
['Antibiotics', ' Azithromycin', ' Hydroxychloroquine', ' Anti-oxidants']
['Antibiotics', 'Azithromycin', 'Hydroxychloroquine', 'Anti-oxidants']


Antibiotics,Anti-oxidants,Hydroxychloroquine,Azithromycin


['\uf071 Azithromycin', ' \uf071 NSAIDs', '  Anti tussive and Acytelcystiene ']
['Azithromycin', ' NSAIDs', '  Anti tussive and Acytelcystiene ']
['Azithromycin', 'NSAIDs', 'Anti tussive and Acytelcystiene']


NSAIDs,Azithromycin


['\uf071 Steroids', ' \uf071 Azithromycin', ' \uf071 Non-NSAIDs', ' \uf071 NSAIDs', ' Others ', ' Vit C & Zn']
['Steroids', ' Azithromycin', ' Non-NSAIDs', ' NSAIDs', ' Others ', 

NSAIDs


['\uf071 Steroids', ' \uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 Hydroxychloroquine', ' \uf071 NSAIDs ']
['Steroids', ' Antibiotics', ' Azithromycin', ' Hydroxychloroquine', ' NSAIDs ']
['Steroids', 'Antibiotics', 'Azithromycin', 'Hydroxychloroquine', 'NSAIDs']


Azithromycin,Antibiotics,Steroids,NSAIDs,Hydroxychloroquine


['nan']
['nan']
['nan']





['\uf071 Steroids', ' Aspirin', ' \uf071 Antibiotics', ' \uf071 Azithromycin', ' \uf071 Non-NSAIDs', ' \uf071 NSAIDs', ' Others ', ' Vit C & Zn']
['Steroids', ' Aspirin', ' Antibiotics', ' Azithromycin', ' Non-NSAIDs', ' NSAIDs', ' Others ', ' Vit C & Zn']
['Steroids', 'Aspirin', 'Antibiotics', 'Azithromycin', 'Non-NSAIDs', 'NSAIDs', 'Others', 'Vit C & Zn']


Non-NSAIDs,Others,Azithromycin,Aspirin,Antibiotics,Steroids,NSAIDs


['nan']
['nan']
['nan']





['\uf071 Antibiotics', ' \uf071 NSAIDs', ' \uf071 Omega 3 / DHA / EPA ']
['Antibiotics', ' NSAIDs', ' Omega 3 / DHA / EPA ']
['Antibiotics', 'NSAIDs', 'Omega 3 / DHA

In [58]:
temp=treat["OTC_prescription_alternate_use"].str.get_dummies(sep=",")
#Covid.drop("pre-existing conditions",inplace=True)
treat=pd.concat([treat,temp],axis=1)
temp

,Anti-histamines,Anti-inflammatories,Anti-oxidants,Antibiotics,Apixaban,Aspirin,Azithromycin,Chloroquine,Cilostazol,Clopidogrel,...,Omega 3 / DHA / EPA,Others,Over the counter painkillers,Oxaloacetate,Remsdesevir,Rivaroxaban,Steroids,Ticagrelor,Veklury,Warfarin
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2568,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2569,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
for i in treat.columns:
    print(i)

Unnamed: 0
PatientID
hospitalized
hospitlization_duration
oxygen_support
OTC_prescription_alternate_use
vaccine
Type of the vaccine
vaccine_shots
vaccine_date
infection_post_vaccination
weird_input
Anti-histamines
Anti-inflammatories
Anti-oxidants
Antibiotics
Apixaban
Aspirin
Azithromycin
Chloroquine
Cilostazol
Clopidogrel
Convalescent plasma
Curcumin
Dabigatran
Dalteparin
Direct oral anticoagulants
Edoxaban
Enoxaparin
Fondaparinux
H1 type Antihistamines
H2 type Antihistamines
Heparin
Hydroxychloroquine
Intravenous gamma globulin
Malaria treatments
NSAIDs
Non-NSAIDs
None
Omega 3 / DHA / EPA
Others
Over the counter painkillers
Oxaloacetate
Remsdesevir
Rivaroxaban
Steroids
Ticagrelor
Veklury
Warfarin


In [60]:
treat[temp.columns].fillna(0,inplace=True)

In [61]:
for i in treat.weird_input:
    print(i)

Ascorbic acid - intestinal antiseptic
nan
nan
nan
vit c
nan
nan
Paracetamol
nan
nan
nan
Multivitamins
surbex Z
nan
nan
nan
nan
Vitamin iron
nan
nan
vitamin c
nan
Panadol + vitamin c + zink
nan
nan
nan
nan
nan
vitamin c
Vitamin and Zinc Supplements
nan
nan
nan
nan
nan
Multivitamins
nan
nan
vitamins
nan
Multi vitamins
Surbex Z multivitamins and Vitamin D capsule
nan
nan
nan
nan
nan
nan
zinc and vit. c
nan
Inhalators for asthma and COPD
Vitamins like B12
dont remeber
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Zinc & vit.c tablets  .... Anti-cough syrup
nan
nan
nan
nan
nan
nan
nan
nan
nan
Panadol
nan
nan
nan
nan
nan
nan
nan
nan
nan
magnesium vitamine D vitamin B12 cortisone
nan
nan
nan
nan
targofloxin
nan
targofloxin
nan
nan
clexane
nan
Vitamins
vitamins
nan
nan
nan
nan
nan
nan
nan
i dont remember
nan
nan
nan
nan
Veklury (an antiviral medicine used to treat coronavirus disease
zinc and vitamin C
nan
nan
nan
nan
nan
nan
nan
nan
nan
vitamin c
nan
nan
nan
nan
Vitamin supplements
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Vitamin C
nan
nan
nan
nan
nan
nan
nan
montelukast sodium
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
antiemetic+zinc+vit c+antiparasitic
vit c + zinc
nan
nan
nan
nan
nan
esomeprazole
vit c
vit c + zinc
nan
nan
nan
nan
nan
nan
Vitamin C
nan
Antitussive+vit c
nan
vit c + zinc
nan
nan
nan
vit c + d
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Natural Herbal drinks and vitamins
nan
nan
nan
nan
nan
nan
nan
nan
nan
vit. C
nan
nan
nan
nan
nan
nan
nan
vit c
zinc+vit c+antiparasitic
nan
nan
vit c + d + lactoferrin
nan
nan
nan
nan
nan
nan
nan
nan
Antitussive + vit c + zinc
vit c + zinc
nan
nan
vitamins
Vit c + zinc + antiparasitic + lactoferrin
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Vitamin C
vit c + zinc +
Antitussive
vit c + antiviral
nan
Vitamins
zinc
vit c + zinc
nan
nan
vit c + zinc + Levodropropizine
nan
nan
Itop

In [62]:
treat.weird_input=treat.weird_input.str.lower().str.strip()

In [63]:
treat.weird_input.value_counts()[:10]

nan              177
vitamin c         36
zinc              30
paracetamol       21
vitamins          16
vit c             14
ivermectin        12
multivitamins     11
vit c + zinc      10
vitc and zinc      7
Name: weird_input, dtype: int64

In [64]:
treat["Vitamin C"]=0
treat["Zinc"]=0
for e,i in enumerate(treat.weird_input):
    if "vit" and "c" in str(i):
        treat.loc[e,"Vitamin C"]==1
    if "z" in str(i):
        treat.loc[e,"Zinc"]=1
    elif "paracetamol" in str(i):
        treat.loc[e,"NSAIDs"]=1

        

In [65]:
treat.drop("weird_input",axis=1,inplace=True)

In [66]:
treat.to_excel("treatment_related_dummies.xlsx",index=False)

In [67]:
treat

,Unnamed: 0,PatientID,hospitalized,hospitlization_duration,oxygen_support,OTC_prescription_alternate_use,vaccine,Type of the vaccine,vaccine_shots,vaccine_date,...,Over the counter painkillers,Oxaloacetate,Remsdesevir,Rivaroxaban,Steroids,Ticagrelor,Veklury,Warfarin,Vitamin C,Zinc
0,0,0, No,NaN,NaN,"Antibiotics,Aspirin,Anti-histamines,NSAIDs",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,NaN,2021-11-15,...,0,0,0,0,0,0,0,0,0,0
1,1,1, No,NaN, No,"Anti-histamines,Antibiotics",Yes, COVID-19 Vaccine AstraZeneca (AZD1222); also...,NaN,NaT,...,0,0,0,0,0,0,0,0,0,0
2,2,2, No,NaN, No,None,Yes, CoronaVac (Sinovac),NaN,2021-04-03,...,0,0,0,0,0,0,0,0,0,0
3,3,3, No,NaN, No,Azithromycin,Yes, CoronaVac (Sinovac),NaN,2021-04-19,...,0,0,0,0,0,0,0,0,0,0
4,4,4, No,NaN, No,"Curcumin,Others,Azithromycin,NSAIDs",Yes, CoronaVac (Sinovac),NaN,2021-02-13,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,2566,2566,NaN,NaN,NaN,,NaN,NaN,NaN,NaT,...,0,0,0,0,0,0,0,0,0,0
2567,2567,2567, Yes,NaN," Yes, nasal cannula","Enoxaparin,NSAIDs",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2021-09-01,...,0,0,0,0,0,0,0,0,0,0
2568,2568,2568, Yes,NaN," Yes, I was intubated","Steroids,Anti-histamines,Enoxaparin",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2021-07-11,...,0,0,0,0,1,0,0,0,0,0
2569,2569,2569, No,NaN, I was not hospitalized,"Aspirin,H1 type Antihistamines,Antibiotics",Yes," Comirnaty (BNT162b2) ( Pfizer, BioNTech; Fos...",NaN,2020-12-20,...,0,0,0,0,0,0,0,0,0,0


## 4.The Depaul Symptoms Questionnaire 

In [11]:
depaul=pd.read_excel("depaul_symp.xlsx",engine="openpyxl")

In [12]:
depaul.head()

,PatientID,persistent_fatigue,headaches,fatigue_duration,MEC_diagnosis,MEC_diagnosis_year,diagnosis_type,diagnosis_by,current_MEC_diagnosis,family_MEC_diagnosis
0,0,No,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1,No,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,2,Not having a problem with fatigue/energy,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,3,No,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,4,No,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [13]:

full_report(depaul)

NameError: name 'full_report' is not defined

ok not much work just a few wrong datatypes 

In [14]:
names=depaul.columns
for i in names:
    print(i,"\n")

PatientID 

persistent_fatigue 

headaches 

fatigue_duration 

MEC_diagnosis 

MEC_diagnosis_year 

diagnosis_type 

diagnosis_by 

current_MEC_diagnosis 

family_MEC_diagnosis 



In [432]:
## let us rename some columns for easier wrangling:
depaul.drop("Unnamed: 0",inplace=True,axis=1)
depaul.rename(columns={names[2]:"persistent_fatigue",
                       names[3]:"headaches",
                       names[4]:"fatigue_duration",
                       names[5]:"MEC_diagnosis",
                       names[6]:"MEC_diagnosis_year",
                       names[7]:"diagnosis_type",
                       names[8]:"diagnosis_by",
                       names[9]:"current_MEC_diagnosis",
                       names[10]:"family_MEC_diagnosis"}, inplace=True)
depaul.columns

Index(['PatientID', 'persistent_fatigue', 'headaches', 'fatigue_duration',
       'MEC_diagnosis', 'MEC_diagnosis_year', 'diagnosis_type', 'diagnosis_by',
       'current_MEC_diagnosis', 'family_MEC_diagnosis'],
      dtype='object')

In [437]:
### wrong datatypes
depaul.PatientID=depaul.PatientID.astype(str)
depaul.MEC_diagnosis_year=pd.to_datetime(depaul.MEC_diagnosis_year,errors="coerce")

In [438]:
##save 

depaul.to_excel("depaul_symp.xlsx",index=False)

## 5. MOS Survey data

In [16]:
symp_2

,PatientID,1. Fatigue/extreme tiredness [Frequency],1. Fatigue/extreme tiredness [Severity],"2. Next day soreness or fatigue after non-strenuous, everyday activities [Frequency ]","2. Next day soreness or fatigue after non-strenuous, everyday activities [Severity]",3. Minimum exercise makes you physically tired [Frequency],3. Minimum exercise makes you physically tired [Severity],4. Feeling unrefreshed after you wake up in the morning [Frequency],4. Feeling unrefreshed after you wake up in the morning [Severity],5. Pain or aching in your muscles [Frequency],...,41.\tDid you have a lot of energy?,42.\tHave you felt down-hearted and blue?,43.\tDid you feel worn out?,44.\tHave you been a happy person?,45.\tDid you feel tired?,"46.\tDuring the past 4 weeks, how much of the time has your physical health or emotional problems interfered with your social activities (like visiting with friends, relatives, etc.)?",47.\tI seem to get sick a little easier than other people,48.\tI am as healthy as anybody I know,49.\tI expect my health to get worse,50.\tMy health is excellent
0,0,2.0,2.0,1.0,1.0,1.0,1.0,4.0,3.0,2.0,...,A Little of the Time,Most of the Time,All of the Time,None of the Time,All of the Time,All of the time .................................,Definitely False,Mostly True,Don’t Know,Don’t Know
1,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,Some of the Time,A Good Bit of the Time,A Good Bit of the Time,Some of the Time,A Little of the Time,Some of the time ................................,Don’t Know,Definitely False,Don’t Know,Mostly False
2,2,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Some of the Time,Some of the Time,A Little of the Time,Some of the Time,Some of the Time,Some of the time ................................,Mostly True,Mostly True,Definitely False,Mostly True
3,3,2.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,...,A Good Bit of the Time,A Good Bit of the Time,A Good Bit of the Time,A Good Bit of the Time,Some of the Time,Most of the time ................................,Don’t Know,Mostly False,Don’t Know,Don’t Know
4,4,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,...,Most of the Time,A Little of the Time,A Little of the Time,Most of the Time,Some of the Time,None of the time ................................,Definitely False,Definitely True,Definitely False,Mostly True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,2566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2567,2567,3.0,2.0,1.0,1.0,3.0,3.0,4.0,2.0,2.0,...,Some of the Time,Most of the Time,Some of the Time,A Little of the Time,Some of the Time,Some of the time ................................,Definitely True,Mostly False,Mostly True,Mostly False
2568,2568,2.0,2.0,2.0,1.0,3.0,3.0,2.0,1.0,4.0,...,None of the Time,Some of the Time,Some of the Time,A Little of the Time,Most of the Time,Some of the time ................................,Mostly True,Mostly False,Definitely True,Mostly False
2569,2569,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,3.0,...,A Little of the Time,Some of the Time,Some of the Time,Most of the Time,A Good Bit of the Time,Some of the time ................................,Don’t Know,Mostly True,Don’t Know,Mostly True


In [17]:
mos=pd.read_excel("mos_symp.xlsx",engine="openpyxl")
mos.head()

,PatientID,mos_0,mos_1,mos_2,mos_3,mos_4,mos_5,mos_6,mos_7,mos_8,...,mos_54,mos_55,mos_56,mos_57,mos_58,mos_59,mos_60,mos_61,mos_62,mos_63
0,0,2.0,2.0,1.0,1.0,1.0,1.0,4.0,3.0,2.0,...,A Little of the Time,Most of the Time,All of the Time,None of the Time,All of the Time,All of the time .................................,Definitely False,Mostly True,Don’t Know,Don’t Know
1,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,Some of the Time,A Good Bit of the Time,A Good Bit of the Time,Some of the Time,A Little of the Time,Some of the time ................................,Don’t Know,Definitely False,Don’t Know,Mostly False
2,2,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,Some of the Time,Some of the Time,A Little of the Time,Some of the Time,Some of the Time,Some of the time ................................,Mostly True,Mostly True,Definitely False,Mostly True
3,3,2.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,...,A Good Bit of the Time,A Good Bit of the Time,A Good Bit of the Time,A Good Bit of the Time,Some of the Time,Most of the time ................................,Don’t Know,Mostly False,Don’t Know,Don’t Know
4,4,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,...,Most of the Time,A Little of the Time,A Little of the Time,Most of the Time,Some of the Time,None of the time ................................,Definitely False,Definitely True,Definitely False,Mostly True


In [5]:
full_report(mos)

Missing Values
 Unnamed: 0                                                                                                                                                                                     0
PatientID                                                                                                                                                                                      0
1. Fatigue/extreme tiredness [Frequency]                                                                                                                                                     685
1. Fatigue/extreme tiredness [Severity]                                                                                                                                                      714
2. Next day soreness or fatigue after non-strenuous, everyday activities  [Frequency ]                                                                                                       806
                   

nothing to do really

In [20]:
# there are many variables that are outputted from checkbox question so we gotta deal with those
mos=symp_2.copy()
mos.PatientID=mos.PatientID.astype(str)
#mos.drop('Unnamed: 0',inplace=True,axis=1)
### a quick way to rename all columns
col_names={}
for e,col in enumerate(mos.columns[1:]):
    col_names[col]=f"mos_{e}"

    
## let us rename some columns for easier wrangling:
#mos.rename(columns=col_names,inplace=True)
#mos.columns
mos_dict=col_names

In [34]:
mos.to_excel("mos_symp.xlsx",index=False)

## 6. Patient Health Quality Survey 

In [7]:
phq=pd.read_excel("phq_symp.xlsx",engine="openpyxl")
phq.head()

,Unnamed: 0,PatientID,1)\tLittle interest or pleasure in doing things?,"2)\tFeeling down, depressed, or hopeless?","3)\tTrouble falling or staying asleep, or sleeping too much?",4)\tFeeling tired or having little energy?,5)\tPoor appetite or overeating?,6)\tFeeling bad about yourself - or that you are a failure or have let yourself or your family down?,"7)\tTrouble concentrating on things, such as reading the newspaper or watching television?",8)\tMoving or speaking so slowly that other people could have noticed? Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual?,"9)\tThoughts that you would be better off dead, or of hurting yourself in some way?Not at all\tSeveral days\tMore than half the days\tNearly every day"
0,0,0,Nearly every day,Nearly every day,Nearly every day,Nearly every day,Nearly every day,More than half the days,More than half the days,More than half the days,Several days
1,1,1,More than half the days,More than half the days,More than half the days,More than half the days,Several days,More than half the days,More than half the days,Several days,Several days
2,2,2,Several days,More than half the days,More than half the days,Several days,Not at all,Several days,Nearly every day,Several days,Not at all
3,3,3,Several days,More than half the days,More than half the days,Several days,Nearly every day,More than half the days,More than half the days,Not at all,Not at all
4,4,4,Not at all,Not at all,Not at all,Not at all,Not at all,Not at all,Not at all,Not at all,Not at all


In [31]:
# there are many variables that are outputted from checkbox question so we gotta deal with those

phq.PatientID=phq.PatientID.astype(str)
#phq.drop('Unnamed: 0',inplace=True,axis=1)
### a quick way to rename all columns
col_names={}
for e,col in enumerate(phq.columns[1:]):
    col_names[col]=f"phq_{e}"

    
## let us rename some columns for easier wrangling:
phq.rename(columns=col_names,inplace=True)
phq.columns

Index(['PatientID', 'phq_0', 'phq_1', 'phq_2', 'phq_3', 'phq_4', 'phq_5',
       'phq_6', 'phq_7', 'phq_8'],
      dtype='object')

In [32]:
phq.to_excel("phq_symp.xlsx",index=False)

## 7. Other symptoms

In [21]:
sympy_4

,PatientID,1.\tWere you given any of these diagnoses for any of your symptoms?,Was this diagnosis?.1,2.\tHave you experienced any MEMORY RELATED SYMPTOMS since the start of your COVID-19 illness? *,3.\tWhich of the following memory symptoms have you experienced since the start of your COVID-19 illness? *,"4.\tHave you experienced issues with BRAIN FOG (inability to focus, think clearly, plan, process, understand, and maintain a coherent stream of thought; abnormally slow or fast thoughts) since the start of your COVID-19 illness? *",5.\tWhich of the following brain fog/cognitive functioning symptoms have you experienced since the start of your COVID-19 illness? *,"6.\tCompared to how you felt before COVID, have you experienced an increase in any of the following? *",7.\tHave you experienced any issues with SPEECH AND LANGUAGE since the start of your COVID-19 illness? *,8.\tWhich of the following speech and language symptoms have you experienced since the start of your COVID-19 illness? *,...,46.\tDo you believe you have or have had a mental health condition that has not been diagnosed?,"47.\tIf you answered yes to either question above, Which of the following have you experienced? (check all that apply)",49.\tHave you ever experienced vertigo/dizziness before COVID-19 diagnosis?,51.\tHave you started experiencing dizziness or vertigo after diagnosis of COVID-19?,"52.\tIf YES, please describe the characteristics of your symptoms",53.\t Indicate the severity of your vertigo/dizziness (0–10),54.\t Have you ever experienced tinnitus before COVID-19 diagnosis?,55. Have you started experiencing tinnitus after diagnosis of COVID-19?,"56. If yes, please specify the characteristics of your tinnitus",57.\tDo you suffer from migraine? (yes/no)
0,0,NaN,NaN,No,NaN,No,I did NOT have any Brain Fog symptoms,None of the above,No,NaN,...,Yes,Anxiety Disorder,No,No,instability,NaN,No,No,NaN,No
1,1,NaN,After COVID-19 infection,Yes,Short-term memory loss (memory that lasts ~30 ...,Yes,Difficulty with executive functioning (plannin...,"Irritability, Anger",No,NaN,...,No,Anxiety Disorder,No,No,violent vertigo attacks,2.0,No,No,continuous,No
2,2,Migraine,Unrelated to COVID,No,NaN,No,NaN,None of the above,No,NaN,...,Yes,"Depression, Anxiety Disorder, Personality Diso...",No,No,chronic dizziness,5.0,No,No,NaN,Yes
3,3,Noppe,Unrelated to COVID,No,Nothing,No,Poor attention or concentration,"Difficulty controlling your emotions, Mood swi...",No,Nothing,...,Yes,Anxiety Disorder,No,Yes,chronic dizziness,1.0,Yes,No,occasional,No
4,4,Irritable bowel syndrome,After COVID-19 infection,No,NaN,No,I did NOT have any Brain Fog symptoms,None of the above,No,NaN,...,Yes,Depression,Yes,Yes,chronic dizziness,6.0,No,No,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,2566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2567,2567,"Polyneuropathy, Encephalopathy, peripheric",After COVID-19 infection,Yes,Long-term memory loss (long-term memory can be...,Yes,Difficulty with executive functioning (plannin...,"Difficulty controlling your emotions, Depressi...",No,None,...,Yes,"Depression, Anxiety Disorder, Post-traumatic s...",Yes,Yes,chronic dizziness,5.0,No,No,occasional,No
2568,2568,Myalgic Encephalomyelitis / Chronic Fatigue Sy...,After COVID-19 infection,Yes,Short-term memory loss (memory that lasts ~30 ...,Yes,"Slowed thoughts, Poor attention or concentration","Difficulty controlling your emotions, Depressi...",No,NaN,...,Yes,"Anxiety Disorder, Eating Disorder",No,Yes,"chronic dizziness, instability",7.0,No,No,NaN,No
2569,2569,None,NaN,Yes,Short-term memory loss (memory that lasts ~30 ...,Yes,"Difficulty problem-solving or decision-making,...",Mood swings,No,NaN,...,Yes,Anxiety Disorder,No,Yes,instability,2.0,Yes,No,NaN,No


In [13]:
#other=pd.read_excel("other_symp.xlsx",engine="openpyxl")
#other.head().T

In [25]:
other_dict=col_names

In [23]:
# there are many variables that are outputted from checkbox question so we gotta deal with those
other=sympy_4
other.PatientID=other.PatientID.astype(str)
#other.drop('Unnamed: 0',inplace=True,axis=1)
### a quick way to rename all columns
col_names={}
for e,col in enumerate(other.columns[1:]):
    col_names[col]=f"other_{e}"

    
## let us rename some columns for easier wrangling:
other.rename(columns=col_names,inplace=True)
other.columns

Index(['PatientID', 'other_0', 'other_1', 'other_2', 'other_3', 'other_4',
       'other_5', 'other_6', 'other_7', 'other_8', 'other_9', 'other_10',
       'other_11', 'other_12', 'other_13', 'other_14', 'other_15', 'other_16',
       'other_17', 'other_18', 'other_19', 'other_20', 'other_21', 'other_22',
       'other_23', 'other_24', 'other_25', 'other_26', 'other_27', 'other_28',
       'other_29', 'other_30', 'other_31', 'other_32', 'other_33', 'other_34',
       'other_35', 'other_36', 'other_37', 'other_38', 'other_39', 'other_40',
       'other_41', 'other_42', 'other_43', 'other_44', 'other_45', 'other_46',
       'other_47', 'other_48', 'other_49', 'other_50', 'other_51', 'other_52',
       'other_53', 'other_54'],
      dtype='object')

In [58]:
other.head().T

,0,1,2,3,4
PatientID,0,1,2,3,4
other_0,NaN,NaN,Migraine,Noppe,Irritable bowel syndrome
other_1,NaN,After COVID-19 infection,Unrelated to COVID,Unrelated to COVID,After COVID-19 infection
other_2,No,Yes,No,No,No
other_3,NaN,Short-term memory loss (memory that lasts ~30 ...,NaN,Nothing,NaN
other_4,No,Yes,No,No,No
other_5,I did NOT have any Brain Fog symptoms,Difficulty with executive functioning (plannin...,NaN,Poor attention or concentration,I did NOT have any Brain Fog symptoms
other_6,None of the above,"Irritability, Anger",None of the above,"Difficulty controlling your emotions, Mood swi...",None of the above
other_7,No,No,No,No,No
other_8,NaN,NaN,NaN,Nothing,NaN


In [59]:
#full_report(other)

mmmmm.... ok here's what we are going to do:
* for questions with an "other:" option where people wrote a mess of answers we will remove these handtyped answers and deal with them indvidually 
* for the headaches question where each option already has many commas we will remove the commas 
* we will then create dummy variables for all the checkbox questions but on collab as my laptop is too old for this 

In [6]:
columns_checkboxes=["other_0","other_3","other_5","other_6","other_8","other_10","other_12","other_15",
                   "other_16","other_18","other_19","other_21","other_22","other_23","other_25",
                   "other_27","other_29","other_31","other_32","other_34","other_36","other_38",
                   "other_40","other_42","other_46","other_49"]

c=[]
for i in other.columns:
    for v in other[i]:
        if "," in str(v):
            c.append(i)
            break


In [7]:
### before dealing with checkboxes, there are many written inputs that we shall deal with 

columns_with_weird_inputs=["other_0","other_3","other_5","other_6","other_8","other_10"
                          ,"other_12","other_15","other_18","other_29","other_36"]


known_values_ofweird_columns=["Guillain-Barre Syndrome",
"Small fiber neuropathy Autonomic neuropathy",
"Polyneuropathy",
"Neuralgia (please include the type of neuralgia in the text box)",
"Antiphospholipid Syndrome, viral-induced or autoimmune",
"Sarcoidosis",
"Stroke (please include the type of stroke in the text box)",
"Demyelinating lesions",
"POTS",
"Encephalopathy",
"Encephalitis (please include the type of encephalitis in the text box)",
"Meningoencephalitis",
"Meningitis",
"Acute Disseminated Encephalomyelitis",
"Acute myelitis",
"Ophthalmo-paresis",
"Psychiatric Diagnosis",
"Migraine",
"Motor Peripheral or Cranial Neuropathies",
"Posterior reversible encephalopathy syndrome",
"Myasthenia",
"Thrombotic microangiopathy",
"Tapia Syndrome",
"Epilepsy",
"Traumatic Brain Injury (TBI) or TBI-like symptoms",
"Myalgic Encephalomyelitis / Chronic Fatigue Syndrome (ME/CFS)",
"Cranial nerve involvement",
"Macular hole",
"Costochondritis",
"Blood clot",
"Myocarditis",
 
"Short-term memory loss (memory that lasts ~30 seconds, i.e.remembering a phone number before writing it down, or forgetting you’re in the middle of a task)",
 
"Long-term memory loss (long-term memory can be anything from remembering yesterday, forgetting you’ve done a task, forgetting recently learned information, or forgetting your third-grade experience)",
 
"Not being able to make new memories",
 
"Forgetting how to do routine tasks (tying your shoelaces, washing your hands) None of the above",
 
"Difficulty with executive functioning (planning, organizing, figuring out the sequence of actions, abstracting)",
 
"Agnosia (failure to recognize or identify objects despite intact sensory functioning)",
 
"Difficulty problem-solving or decision-making",
 
"Difficulty thinking",

"Thoughts moving too quickly",
 
"Slowed thoughts",

"Poor attention or concentration",
 
"I did NOT have any Brain Fog symptoms",
 
"Difficulty controlling your emotions",
"Lack of inhibition (difficulty controlling your behavior)",
"Irritability",
"Anger",
"Impulsivity (acting on a whim without self-control)",
"Aggression",
"Euphoria (a feeling or state of intense excitement and happiness)",
"Delusions",
"Depression",
"Apathy (lack of feeling, emotion, interest, or concern)",
"Suicidality",
"Mood swings",
"Anxiety",
"Mania (abnormally elevated/excited mood, decreased need for sleep, occasionally with delusions) Hypomania (a milder form of mania)",
"Tearfulness",
"Sense of doom",
"None of the above",
 
 
 "Difficulty finding the right words while speaking/writing",
"Difficulty communicating verbally",
"Difficulty speaking in complete sentences",
"Speaking unrecognizable words",
"Difficulty communicating in writing",
"Difficulty processing/understanding what others say",
"Difficulty reading/processing written text",
"(If applicable) changes to your non-primary (second/third) language skills",
 
'Headaches, at the base of the skull',
'Headaches, in the temples',
'Headaches, behind the eyes',
'Headaches, diffuse (entire brain)',
'Headaches/pain after mental exertion',
'Sensation of brain warmth/"on fire"',
'Sensation of brain pressure',
'Migraines',
'None of the above',
 
 
"Loss of smell",
"Phantom smells (imagining/hallucinating smells - smelling things that aren't there)",
"Heightened sense of smell",
"Altered sense of smell",
"Loss of taste",
"Phantom taste (imagining/hallucinating tastes - tasting things when there's nothing in your mouth)",
"Heightened sense of taste",
"Altered sense of taste",
"None of the above",
                
                
                
"Lucid dreams (dreams where you are aware you are dreaming or have some control over what you dream about)",
"Vivid dreams",
"Nightmares",
"Insomnia",
"Night sweats",
"Restless leg syndrome",
"Awakened by feeling like you couldn’t breathe",
"Sleep apnea",
                
                
"Visual (seeing) Hallucinations",
"Auditory (hearing) Hallucinations",
"Tactile (touch) Hallucinations",
"Hallucinations",
                
"None of the below skin and allergy symptoms apply to me",
"Skin and Allergy Symptoms",
"Peeling skin",
"Petechiae",
"COVID toes (discoloration, swelling, painful, or blistering toes)",
"Dermatographia",
"New allergies (food, chemical, environmental, etc)",
"Skin rashes",
                
                
"Vision symptoms - Blurred vision",
"Vision symptoms - Double vision",
"Vision symptoms - Sensitivity to light",
"Vision symptoms - Tunnel vision",
"Vision symptoms - Total loss of vision",
"Eye pressure or pain",
"Pink eye (conjunctivitis)",
"Bloodshot eyes",
"Dry eyes",
"Redness on the outside of eyes",
"Floaters",
"Seeing things in your peripheral vision",
"None of the above",

]

In [10]:
from tqdm import tqdm
for col in tqdm(columns_with_weird_inputs):
    other[f"{col}_weird"]=np.nan
    for e,v in enumerate(other[col]):
        v=str(v)
        if not any(elem in v for elem in known_values_ofweird_columns):
            other.loc[e,f"{col}_weird"]=v
            other.loc[e,col]=np.nan



        

100%|███████████████████████████████████████████| 11/11 [00:11<00:00,  1.01s/it]


In [11]:
other.shape

(2571, 67)

In [12]:
#other.to_excel("other_symp2.xlsx",index=False)

##### CHECKPOINT

In [8]:
other=pd.read_excel("other_symp2.xlsx")

In [9]:
for e,col in enumerate(columns_checkboxes):
    print(e,"\n",col,"\n",other[col].value_counts(),"\n \n")

0 
 other_0 
 Migraine                                                                                     182
Psychiatric Diagnosis                                                                         32
None of the above                                                                             32
Blood clot                                                                                    15
Psychiatric Diagnosis, Migraine                                                                9
                                                                                            ... 
Ophthalmo-paresis, type2 Diabetes                                                              1
Migraine, Motor Peripheral or Cranial Neuropathies, Myasthenia, Cranial nerve involvement      1
Migraine, Myasthenia, Myocarditis                                                              1
Psychiatric Diagnosis, Migraine, Motor Peripheral or Cranial Neuropathies, Myasthenia          1
Polyneuropathy, 

In [10]:
def get_key(val,my_dict):
    for key, value in my_dict.items():
         if val == value:
                return key
 
    return "key doesn't exist"


In [11]:
 for col in columns_checkboxes:
        print(col,get_key(col,col_names),"\n")

NameError: name 'col_names' is not defined

In [14]:
shall_remove_brackets=["other_3","other_5","other_6","other_12","other_15",
                      "other_34","other_42"]
dealt_with=["other_10","other_19","other_22","other_38","other_29"]

In [9]:
## remove data between brackets 
def remove_brackets(string):
    
    while "(" in str(string) and ")" in str(string):
        br1=string.index("(")
        br2=string.index(")")
        #print(br1,br2,string[br1:br2+1])
        string=string.replace(string[br1:br2+1],"")
    return string
print(remove_brackets("bla bla bla (hahahaha,ffff) (4444),44,55,66(i)"),remove_brackets(None))

bla bla bla  ,44,55,66 None


In [16]:
other2=other.copy()
for e,col in enumerate(shall_remove_brackets):
    print(e,"/",len(shall_remove_brackets))
    other2[col]=other2[col].apply(remove_brackets)


0 / 7
1 / 7
2 / 7
3 / 7
4 / 7
5 / 7
6 / 7


In [17]:
#other_10
other2.other_10=other2.other_10.str.replace("Headaches,","Headaches-")

In [18]:
#other_19
other2.other_19=other2.other_19.str.replace("Skin sensations: burning, tingling, or itchiness without rash Numbness/loss of sensation"
                                          ,"skin sensations")

other2.other_19=other2.other_19.str.replace("numbness,","numbness-")



In [19]:
#other_22
other2.other_22=other2.other_22.str.replace("\(high heart rate, >90 beats per minute\)",">90 bpm")
other2.other_22=other2.other_22.apply(remove_brackets)


In [20]:
#other_38
other2.other_38=other2.other_38.str.replace("Sexual dysfunction \(difficulty maintaining erection, vaginal dryness,"
                                          ,"Sexual dysfunction")




In [21]:
#other_29
other2.other_29=other2.other_29.str.replace("\(writing","writing")
other2.other_29=other2.other_29.str.replace("scratched\)","scratched")
other2.other_29=other2.other_29.apply(remove_brackets)


In [22]:

for e,col in enumerate(columns_checkboxes):
    print(e,"\n",col,"\n",other2[col].value_counts(),"\n \n")

0 
 other_0 
 Migraine                                                                                     182
Psychiatric Diagnosis                                                                         32
None of the above                                                                             32
Blood clot                                                                                    15
Psychiatric Diagnosis, Migraine                                                                9
                                                                                            ... 
Ophthalmo-paresis, type2 Diabetes                                                              1
Migraine, Motor Peripheral or Cranial Neuropathies, Myasthenia, Cranial nerve involvement      1
Migraine, Myasthenia, Myocarditis                                                              1
Psychiatric Diagnosis, Migraine, Motor Peripheral or Cranial Neuropathies, Myasthenia          1
Polyneuropathy, 

In [24]:
from tqdm import tqdm
other3=other2.copy()
for col in tqdm(columns_checkboxes):
    #print(col)
    temp=other3[col].str.get_dummies(sep=",")
    col_rename={}
    for name in temp.columns:
        col_rename[name]=f"{name}_{col}"
    temp.rename(columns=col_rename,inplace=True)
    other3.drop(col,inplace=True,axis=1)
    other3[temp.columns]=temp
    #print(other3.shape)
    del temp
other3.shape

100%|███████████████████████████████████████████| 26/26 [00:00<00:00, 26.79it/s]


(2571, 548)

In [25]:
other.shape

(2571, 67)

In [26]:
##calculate sparsity
a = (other3.to_numpy() == 0).mean()
print (a)


0.8915202412080846


In [27]:
weird_columns=[]
for col in other3.columns:
    if "weird" in col:
        weird_columns.append(col)
        other3[col]=other3[col].str.lower().str.strip()
        print(col,'\n')
        print(other3[col].value_counts())
        other3.drop(col,inplace=True,axis=1)

other_0_weird 

none                                                    113
no                                                       25
nothing                                                   9
none of above                                             6
non                                                       5
                                                       ... 
no thing                                                  1
not have any of these symptoms                            1
mitral regurgetation and decreased ejection fraction      1
backer cyst                                               1
mitral valve prolapse                                     1
Name: other_0_weird, Length: 101, dtype: int64
other_3_weird 

none                                                                                                                                                                                       85
no                                                                     

In [35]:
other3.to_excel("other_symp_dummies.xlsx",engine="openpyxl",index=False)

In [28]:
for col in other3.columns:
    print("/n",col)

/n PatientID
/n other_1
/n other_2
/n other_4
/n other_7
/n other_9
/n other_11
/n other_13
/n other_14
/n other_17
/n other_20
/n other_24
/n other_26
/n other_28
/n other_30
/n other_33
/n other_35
/n other_37
/n other_39
/n other_41
/n other_43
/n other_44
/n other_45
/n other_47
/n other_48
/n other_50
/n other_51
/n other_52
/n other_53
/n other_54
/n  Acute myelitis_other_0
/n  Autoimmune encephalitis _other_0
/n  Blood clot_other_0
/n  Bone problems. Arthritis _other_0
/n  Costochondritis_other_0
/n  Cranial nerve involvement_other_0
/n  Encephalopathy_other_0
/n  Epilepsy_other_0
/n  Gastroesophageal reflux disease_other_0
/n  Irritable bowel syndrome_other_0
/n  Known case of epilepsy but long time no attach of seizure but developed provoked seizure with Covid infection due to the fever .   Also developed Decrease urination -Given  diagnosis of AKI due to rhabdomyolysis_other_0
/n  Macular hole_other_0
/n  Meningitis_other_0
/n  Meningoencephalitis_other_0
/n  Migraine_other_0